In [1]:
import numpy as np
# from google.colab import drive

# This will prompt for authorization......
# drive.mount('/content/drive')

In [2]:
#path = '/content/drive/My Drive/Colab Notebooks/Music/ADEHack2018LSTM/queen/'
path = "Documents/GitHub/tensorflow_1.X_playground/time_series/models/queen/"
path = 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\'

In [3]:
#!pip install music21

In [4]:
from music21 import instrument, note, stream, chord, converter

instr = instrument.SnareDrum()
#chkpt = 'new-weights-improvement-12-0.1794-bigger.hdf5'
import glob
import os

list_of_files = glob.glob(path+'checkpoints\\*.hdf5') # gets latest checkpoint
chkpt = max(list_of_files, key=os.path.getctime)

fl = ''

import glob
import os

#list_of_files = glob.glob(path+'created\\*.mid') # gets latest checkpoint
#fl = max(list_of_files, key=os.path.getctime)


testthis = False

length = 100#100#120#60#60
s_length = 200

In [ ]:
chkpt

'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\checkpoints\\ckpt.hdf5'

In [ ]:
def feed_existing_sample(filename):
    notes = []
    midi = converter.parse(filename)

    print("Parsing %s" % filename)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            
    return notes

In [ ]:
"""exchanges random sampling of seed with a file"""
if testthis == True:
        notes = feed_existing_sample(fl)
        s_length = len(notes)//3
s_length

200

In [ ]:
#list_of_files

In [ ]:

""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open(path+'data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    if testthis == True:
        notes = feed_existing_sample(fl)
        #s_length = len(notes)-1
        
    print(notes[:100])
    sequence_length = s_length
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    
    n_patterns = len(network_input)
    print(n_patterns)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    
    model = Sequential()
    model.add(LSTM(
        250,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(300, return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(300))
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    #print(network_input[start])
    prediction_output = []

    # generate 500 notes
    for note_index in range(length):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instr
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instr
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    song_name='created/NOV18queen3Loss{}Len{}sLen{}.mid'.format(''.join(chkpt.split('-')[3:5]),length,s_length)
    midi_stream.write('midi', fp=path+song_name)
    print('made: '+song_name)
    #midi_stream.write('midi', fp=path+'created/queen3Lossjiz{}Len{}sLen{}aoringinal{}.mid'.format(''.join(chkpt.split('-')[3:5]),length,s_length,fl.split('/')[1].split('.')[1][:5]))
    #print('created/queen2Lossj{}Len{}sLen{}aoringinal{}.mid'.format(''.join(chkpt.split('-')[3:5]),length,s_length,fl.split('/')[1].split('.')[1]))
if __name__ == '__main__':
    generate()

['7.9.2', '4.7', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'E4', 'E4', 'E4', 'E4', 'E4', 'G4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'C5', 'B4', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'G4', 'G4', 'C5', 'C5', 'B-4', 'A4', 'G4', 'C5', 'C5', 'C5', 'C#5', 'D5', 'D5', 'G4', 'A4', 'A4', 'A4', 'A4', 'B-4', 'B4', 'G2', 'E2', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'B-4', 'B4', 'B4', 'A4', 'A4', 'G4', 'E4', 'B4', 'A4', 'A4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'F#4', 'F#2', 'F#4', 'F#2', 'F#4', 'A4', 'A2', 'A4', 'A2', 'A4', 'A2']
17277
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
made: created/NOV18queen3LossLen100sLen200.mid


In [ ]:
import glob
import os

list_of_files = glob.glob('created/*.mid') # gets latest checkpoint
file = max(list_of_files, key=os.path.getctime)

file

'created\\NOV18queen3LossLen100sLen200.mid'

In [ ]:
import pygame
pygame.init()

pygame.mixer.music.load(file)
pygame.mixer.music.play()
#pygame.event.wait()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
notes1 = ['7.9.2', '4.7', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'E4', 'E4', 'E4', 'E4', 'E4', 'G4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'C5', 'B4', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'G4', 'G4', 'C5', 'C5', 'B-4', 'A4', 'G4', 'C5', 'C5', 'C5', 'C#5', 'D5', 'D5', 'G4', 'A4', 'A4', 'A4', 'A4', 'B-4', 'B4', 'G2', 'E2', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'B-4', 'B4', 'B4', 'A4', 'A4', 'G4', 'E4', 'B4', 'A4', 'A4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'F#4', 'F#2', 'F#4', 'F#2', 'F#4', 'A4', 'A2', 'A4', 'A2', 'A4', 'A2', 'F#4', 'F#2', 'F#4', 'F#4', 'A4', 'F#4', 'B4', 'G2', 'E2', '7.9.2', '4.7', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'G5', '11.2.4.7', 'G5', '11.2.4.7', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', 'B5', '11.2.4.7', 'A5', 'A5', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'E4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'E5', 'E5', 'G5', '11.2.4.7', 'G5', 'G5', '11.2.4.7', 'G5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', 'B5', '11.2.4.7', 'A5', 'A5', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'C6', 'C6', 'B5', 'B5', 'C6', 'C3', 'C6', 'C6', 'C6', 'C3', 'C3', 'C6', 'C6', 'C6', 'C#3', 'C6', 'D6', 'D3', 'D6', 'D6', 'G3', 'D6', 'G5', 'G5', 'G2', 'G5', 'G5', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'B-5', 'B-5', 'A5', 'C3', 'A5', 'G5', 'C#3', 'G5', 'D3', 'G2', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'C#6', 'C#3', 'C#6', 'D6', 'D3', 'D6', 'D6', 'G3', 'D6', 'G5', 'G2', 'G5', 'A5', 'A2', 'A5', 'A5', 'A2', 'A5', 'A5', 'A2', 'A5', 'A5', 'A5', 'B-5', 'B-2', 'B-5', 'B2', 'B5', 'B5', 'G2', 'E2', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'B-4', 'B4', 'B4', 'A4', 'A4', 'G4', 'E4', 'B4', 'A4', 'A4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'F#4', 'F#2', 'F#4', 'F#2', 'F#4', 'A4', 'A2', 'A4', 'A2', 'A4', 'A2', 'F#4', 'F#2', 'F#4', 'F#4', 'A4', 'F#4', 'B4', 'G2', 'E2', '7.9.2', 'E0', 'B0', 'G0', 'B0', '4.7', '10.1', 'G#0', 'B-0', 'B3', 'G0', 'A0', '10.1', 'B2', '5.8', '10.3', 'F#0', '4.7', 'B-2', 'F#4', 'F#3', 'G3', 'E-0', 'B-0', 'B-0', 'E-4', 'G#4', 'G2', 'G0', 'A0', 'F#0', 'G#0', 'E-3', 'G#3', '2.4.5.7.9', '3.6', 'F#4', 'F4', 'C#5', 'B-4', 'F#3', 'F3', 'C#4', 'B-3', 'B-4', 'G#0', 'C0', 'E-5', 'B-3', 'B-1', 'E-4', 'F#2', '7.9', 'A4', 'F#2', 'B4', 'B4', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'E5', 'E5', 'B4', 'B4', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'E5', 'E5', 'G5', 'G5', 'G5', 'G5', 'B4', 'B4', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'E5', 'E5', 'B4', 'B4', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'E5', 'E5', 'G5', 'G5', 'A5', 'A5', 'G5', 'G5', 'A5', 'B5', '7.9.2', 'B5', 'A5', 'A5', 'G5', 'G5', 'A5', 'A5', '4.7', 'G5', 'G5', 'E5', 'E5', 'G5', 'G5', 'E5', 'E5', 'D5', 'D5', 'E5', 'E5', 'D5', 'D5', 'B4', 'B4', 'D5', 'D5', 'B4', 'B4', 'B-4', 'B-4', 'A4', 'A4', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'E4', 'E4', 'D4', 'D4', 'B3', 'B3', 'A3', 'A3', 'G3', 'G3', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'G5', '11.2.4.7', 'G5', '11.2.4.7', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', 'B5', '11.2.4.7', 'A5', 'A5', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'E4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'E5', 'E5', 'G5', '11.2.4.7', 'G5', 'G5', '11.2.4.7', 'G5', '2.4', 'E5', '11.2.4.7', 'E5', '11.2.4.7', 'E5', '2.4', 'E5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', '11.2.4.7', 'B5', 'B5', 'B5', '11.2.4.7', 'A5', 'A5', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'C6', 'C6', 'B5', 'B5', 'C6', 'C3', 'C6', 'C6', 'C6', 'C3', 'C3', 'C6', 'C6', 'C6', 'C#3', 'C6', 'D6', 'D3', 'D6', 'D6', 'G3', 'D6', 'G5', 'G5', 'G2', 'G5', 'G5', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'B-5', 'B-5', 'A5', 'C3', 'A5', 'G5', 'C#3', 'G5', 'D3', 'G2', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'C6', 'C3', 'C6', 'C#6', 'C#3', 'C#6', 'D6', 'D3', 'D6', 'D6', 'G3', 'D6', 'G5', 'G2', 'G5', 'A5', 'A2', 'A5', 'A5', 'A2', 'A5', 'A5', 'A2', 'A5', 'A5', 'A5', 'B-5', 'B-2', 'B-5', 'B2', 'B5', 'B5', 'G2', 'E2', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'E4', 'E4', 'E4', 'E4', 'G4', 'E4', 'A4', 'B-4', 'B4', 'B4', 'A4', 'A4', 'G4', 'E4', 'B4', 'A4', 'A4', 'G4', 'E4', 'E4', 'B4', 'B4', 'B4', 'B4', 'A4', 'F#4', 'F#2', 'F#4', 'F#2', 'F#4', 'A4', 'A2', 'A4', 'A2', 'A4', 'A2', 'F#4', 'F#2', 'F#4', 'F#4', 'A4', 'F#4', 'B4', 'G2', '11.2.4.7', 'E2', '11.2.4.7', '2.4', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '11.2.4.7', '11.2.4.7', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'E4', '11.2.4.7', '11.2.4.7', '2.4', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '2.4', '11.2.4.7', '11.2.4.7', '11.2.4.7', '11.2.4.7', '9.10', '9.10.0.4', '9.0.4', '0.4', 'A3', '11.2', '9.0.4', 'C3', 'C3', 'C3', 'C#3', 'D3', 'G3', 'G2', 'C3', 'C3', 'C3', 'C#3', 'D3', 'G2', 'C3', 'C3', 'C3', 'C#3', 'D3', 'G3', 'G2', 'A2', 'A2', 'A2', 'B-2', 'B2', 'A0', '3.8', 'G#3', 'C4', 'G#3', '5.10', 'B-3', 'C#4', 'B-3', '2.6.9', '2.6', 'D3', 'A1', '2.6.9', '2.6', 'D3', 'A1', '2.6.9', '2.6', 'D3', 'A1', 'A2', 'G2', 'B2', 'F#4', 'E-4', 'B3', '2.6', 'A1', 'B2', '2.6', 'A1', 'B2', 'F#4', '2.6', 'E-4', 'A1', 'B3', 'A2', 'G2', '8.0.3', '2.6', '8.0.3', 'A1', '8.0.3', 'G#2', '2.6', 'A1', 'G#4', '2.6', '8.0', 'A1', '8.0', 'A2', '8.0', 'G2', 'G#2', '0.3', 'G#3', '6.9.1', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', 'F#2', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', '8', '6.9', 'F#2', 'E-4', 'F#2', 'F#2', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', 'G#2', '8.0.3', '6.9', 'F#2', '8.0.3', 'F#2', 'F#2', '8.0.3', '10.2', '8.0.3', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', '8', '6.9', 'F#2', 'E-4', 'F#2', 'F#2', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', '8', '6.9', 'F#2', 'E-4', 'F#2', 'F#2', 'G#4', '10.2', '2.4', '10.1.5', '6.9', '10.2', 'G#2', '10.1.5', '6.9', '10.2', 'G#2', 'A1', '10.1.5', '6.9', 'A1', '2.4', '10.1.4.6', '9.1', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '5.7.10.1', '7.9', '9', '9.10', '10', '6.9', 'C3', 'D3', 'A2', 'A1', '10.1.5', '6.9', '10.2', 'G#2', '10.1.5', 'A1', '10.2', 'G#2', 'A1', '10.1.5', 'A1', '10.1.4.6', '9.1', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '1.4.7.10', '9', '1.4.7.10', '1.4.7.10', '1.4.7.10', '1.4.7.10', '1.4.7.10', '5', '7.9', '5', '5', '5', '7', '9', 'E-3', '3.7.10', '2.6', '3.7.10', '6.9', '3.7.10', '2.6', '3.7.10', '6.9', 'G4', '2.6', 'E-4', '6.9', 'F#2', 'F#2', '2.6.10', 'D3', '2.6', '2.6.10', '6.9', '2.6.10', '2.6', '2.6.10', '6.9', 'F#4', '2.6', 'D4', '6.9', 'F#2', 'F#2', '10.1.5', 'C#3', '2.6', '10.1.5', '6.9', '10.1.5', '2.6', '10.1.5', '6.9', 'B-4', '2.6', '6.9', 'F4', 'F#2', 'F#2', '3.8', 'G#3', '9.1', 'C4', 'G#3', '5.10', 'B-3', '2.6', 'C#4', '2.6', 'B-3', '2.6.9', '2.6', 'D3', '6.9', '2.6.9', '2.6', 'D3', '6.9', '2.6.9', '2.6', 'D3', '6.9', '2.6', '6.9', 'B2', 'F#4', 'E-4', 'B3', '2.6', '6.9', 'B2', '2.6', '6.9', 'B2', 'F#4', '2.6', 'E-4', '6.9', 'B3', '2.6', '6.9', '8.0.3', '2.6', '8.0.3', '6.9', '8.0.3', 'G#2', '2.6', '6.9', 'G#4', '2.6', '8.0', '6.9', '8.0', '2.6', '8.0', '6.9', 'G#2', '0.3', 'G#3', '6.9.1', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', 'F#2', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', '8', '6.9', 'F#2', 'E-4', 'F#2', 'F#2', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', 'G#2', '8.0.3', '6.9', 'F#2', '8.0.3', 'F#2', 'F#2', '8.0.3', '10.2', '8.0.3', 'G#2', 'C4', '6.9', 'F#2', 'G#3', 'F#2', 'C4', '6.9', 'B-2', 'C#4', '10.2', 'G#2', 'F#2', 'B-3', 'F#2', 'C#4', 'F#2', '8', '6.9', 'C#3', 'E-4', 'F3', 'G3', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', 'G2', '2.5.7.10', '9.1', '2.5.7.10', 'E-3', '2.5.7.10', 'E-3', '2.5.7.10', '2.3', '2.5.7.10', 'E-3', 'A1', '2.5.7.10', 'E-3', 'C3', '4.7.10.0', '9.1', '4.7.10.0', 'E-3', '4.7.10.0', 'E-3', '4.7.10.0', '10.2', '0', '2', 'A1', 'A1', 'F2', '5.9.0', '9.1', '5.9.0', 'E-3', '5.9.0', 'E-3', '5.9.0', 'D2', '5.9.0', 'E-3', '5.9.0', '3.9', 'B-2', '10.2.5', '9.1', '10.2.5', 'E-3', '10.2.5', 'E-3', '5.9.0', '2.3', '7.10.2', '5.9.0', 'E-3', '5.9', '3.9', 'G2', '2.5.7.10', '9.1', '2.5.7.10', 'E-3', '2.5.7.10', 'E-3', '2.5.7.10', '2.3', '2.5.7.10', 'E-3', '2.5.7.10', '3.9', 'C3', '4.7.10.0', '9.1', '4.7.10.0', 'E-3', '4.7.10.0', 'E-3', '4.7.10.0', '2.3', '0', '3.9', '2', '3.9', 'F2', '5.9.0', '9.1', '5.9.0', 'E-3', '5.9.0', 'E-3', '5.9.0', '2.3', '5.9.0', 'E-3', 'A1', '5.9.0', '3.9', 'B-2', '10.2.5', '9.1', '10.2.5', 'E-3', '10.2.5', 'E-3', '10.2.5', '2.3', '10.2.5', '3.9', 'A1', '10.2.5', '3.9', '10.2.5', '9.1', '9.10.3', '3.7.10', '3.5.10', '9.1.4', '9.1', '9.10.3', 'G2', '9.10.3', '2.5.9', '9.1', '10.3', '0.4', '9.1', '5.9', '7.9', '4.10', '9.1', '9.2', '9.1', 'C3', 'G#2', '9.0', '9.1', '2.5', '7.9', 'C3', '9.1', '10', '9.1', '0', '2.4', '7.11.2', '9.1', 'E-3', '2.3', '4.7.11', '9.1', 'E-3', '2.3', '5.7.11', '9.1', 'E-3', 'E-3', '7', '7.9', '7', '7', '7', '9', '11', '5.9.0', '2.6', '5.9.0', 'A1', '5.9.0', '2.6', '5.9.0', 'A1', '5.9', '2.6', '5', 'A1', 'F3', 'F#2', 'F3', 'F#2', '0.4', '2.6', '0.4', 'A1', '0.4', '2.6', '0.4', 'A1', '4.9', '2.6', '4', 'A1', 'E3', 'F#2', 'E3', 'F#2', '8.0.3', '2.6', '8.0.3', 'A1', '8.0.3', '2.6', '8.0.3', 'A1', '3.8', '2.6', '3.8', 'A1', '3', 'F#2', '1.3', 'F#2', 'G#3', 'G#4', 'G#2', '9.1', 'G#3', 'C4', 'F#2', 'G#3', 'F#2', 'B-3', 'B-2', 'G#4', '10.2', 'B-3', 'C#4', '10.2', 'B-3', 'D3', '2.6.9', '2.6', 'A1', '2.6.9', '2.6', 'A1', '2.6.9', '2.6', 'A1', '2.6.9', '2.6', 'A1', 'B2', '11.3.6', '2.6', 'A1', 'E-4', '6.11', '2.6', 'A1', 'B3', 'F#4', '2.6', 'E-4', 'A1', 'B3', '2.6', 'A1', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '7', '5.9', '0', '7.9', '6.9', '1.2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '6.9', '2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '1.4', '2.6', '1.4', 'A1', '9', '2.6', '9', 'A1', '1.4', '2.6', '1.4', 'A1', '9', '2.6', '9', 'A1', '6.9', '2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '6.9', '2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '4', '2.6', '4', 'A1', '6.9', '2.6', '6.9', 'A1', '7', '2.6', '7', 'A1', '9', '2.6', '9', 'A1', '6.9', '1.2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '6.9', '2.6', '6.9', 'A1', '2', '2.6', '2', 'A1', '11', '1.2.6', 'A1', '6', '2.6', 'E-4', 'A1', '11', '2.6', 'A1', '6', '2.6', 'E-4', 'A1', '4', '1.2.6', 'A1', '11', '2.6', 'A1', '4', '2.6', 'A1', '11', 'G4', '2.6', 'E4', 'A1', 'F#2', '10.2.4', '6', 'A1', '6', '6.9', 'F#2', '10.2.4', '6', 'A1', '6', '6.9', 'F#2', '10.2.4', '6', 'A1', '6', '6.9', 'F#2', '10.2.4', '6', 'F#2', 'F#2', '10.2.4', '6', 'F#2', 'F#2', '9.1', '6', '6', 'F#2', 'G#3', '7.9', '6', '6', 'F#2', 'B-3', '7.9', '6', '6', 'F#2', 'C#4', '9.1', '6', 'E-4', '7.9', '6', 'E4', '9', '2.4', '10.1.5', '6.9', '10.2', 'G#2', '10.1.5', '6.9', '10.2', 'G#2', 'A1', '10.1.5', '6.9', '2.4', '10.1.4.6', '9.1', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '5.7.10.1', '7.9', '9', '9.10', '10', '6.9', 'C3', 'D3', 'A2', 'A1', '10.1.5', '6.9', '10.2', 'G#2', '10.1.5', '6.9', '10.2', 'G#2', 'A1', '10.1.5', '6.9', 'B-2', '10.1.5', '2.4', '10.1.4.6', '9.1', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '10.1.4.6', '1.4.7.10', '9', '1.4.7.10', '1.4.7.10', '1.4.7.10', '1.4.7.10', '1.4.7.10', '5', '7.9', '5', '5', '5', '7', '9', 'E-3', '3.7.10', '2.6', '3.7.10', '6.9', '3.7.10', '2.6', '3.7.10', '6.9', 'G4', '2.6', 'E-4', '6.9', 'F#2', 'F#2', '2.6.10', 'D3', '2.6', '2.6.10', '6.9', '2.6.10', '2.6', '2.6.10', '6.9', 'F#4', '2.6', 'D4', '6.9', 'F#2', 'F#2', '10.1.5', 'C#3', '2.6', '10.1.5', '6.9', '10.1.5', '2.6', '10.1.5', '6.9', 'B-4', '2.6', '6.9', 'F4', 'F#2', 'F#2', '3.8', 'G#3', '9.1', 'C4', 'G#3', '5.10', 'B-3', '2.6', 'C#4', '2.6', 'B-3', '2.6.9', '2.6', 'D3', '6.9', '2.6.9', '2.6', 'D3', '6.9', '2.6.9', '2.6', 'D3', '6.9', '2.6', '6.9', 'B2', 'F#4', 'E-4', 'B3', '2.6', '6.9', 'B2', '2.6', '6.9', 'B2', 'F#4', '2.6', 'E-4', '6.9', 'B3', '2.6', '6.9', '8.0.3', '2.6', '8.0.3', '6.9', '8.0.3', 'G#2', '2.6', '6.9', 'G#4', '2.6', '8.0', '6.9', '8.0', '2.6', '8.0', '6.9', 'G#2', '0.3', 'G#3', '6.9.1', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', 'F#2', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', '8', '6.9', 'F#2', 'E-4', 'F#2', 'F#2', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', 'G#2', '8.0.3', '6.9', 'F#2', '8.0.3', 'F#2', 'F#2', '8.0.3', '10.2', '8.0.3', 'G#2', 'C4', '6.9', 'F#2', 'G#3', 'F#2', 'C4', '6.9', 'B-2', 'C#4', '10.2', 'G#2', 'F#2', 'B-3', 'F#2', 'C#4', 'F#2', '8', '6.9', 'C#3', 'E-4', 'F3', 'G3', 'G#4', '3.5', 'B-2', 'G#2', '0.3', 'G#3', '6.9', 'G#3', 'F#2', 'C4', 'F#2', 'G#3', '6.9', 'B-2', 'E-4', 'B-3', '10.2', 'G#2', 'B-3', 'F#2', 'C#4', 'F#2', 'B-3', 'F#2', 'G#2', 'G#3', '6.7.9', 'F3', 'E-4', 'E-3', 'C4', 'F3', '8.0.3', 'C5', 'D5', 'B-4', 'C5', 'D5', 'C#5', 'D5', 'E-5', 'F5', 'E-5', 'D5', 'C5', 'C5', 'D5', 'E-5', 'F5', 'E-5', 'D5', 'C5', 'C#5', 'D5', 'D5', 'C5', 'B-4', 'C5', 'D5', 'F5', 'G#5', 'A5', 'G5', 'F#5', 'G5', 'B-5', 'B-5', 'B-5', 'A5', 'B-5', 'G5', 'D5', 'E-5', 'C5', 'F#5', 'G5', 'F5', 'G5', 'G#5', 'G5', 'G5', 'G5', 'G#5', 'G5', 'G5', 'F5', 'F5', 'B-4', 'B-4', 'F5', 'F5', 'G5', 'G5', 'G#5', 'G#5', 'B-5', 'G#5', 'G5', 'F5', 'E-5', 'F5', 'E-5', 'D5', 'E-5', 'B-4', 'B-4', 'B4', 'C#5', 'B4', 'C#5', 'B4', 'C#5', 'B4', 'B-4', 'E6', 'E6', 'A6', 'E6', 'E6', 'A6', 'E6', 'E6', 'E-6', 'E-6', 'E6', 'F#6', 'E6', 'C6', 'C6', 'C6', 'A5', 'F5', 'D5', 'C5', 'C6', 'G#5', 'F5', 'E-5', 'C4', 'B3', 'G#3', 'F4', 'G3', 'D4', '5.8', 'D3', 'G4', '0', 'G#5', 'G5', 'E-5', 'G4', 'B-4', 'G5', 'F5', 'E-5', 'D5', 'B-4', 'G5', 'E-5', 'F5', 'G5', 'E-5', 'B4', 'F#5', 'G5', 'F5', 'F5', 'E-5', 'E-5', 'C5', 'D5', 'E-5', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'A1', 'A1', 'A1', 'A1', 'D1', 'D1', 'D1', 'D1', 'G1', 'G1', 'G1', 'G1', 'D1', 'D1', 'A1', 'A1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'D2', 'C2', 'A1', 'G1', 'F1', 'E1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'A1', 'F1', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'B1', 'A1', 'A1', 'E2', 'G2', 'A2', 'G2', 'F#2', 'F2', 'E2', 'E-2', 'D2', 'C#2', 'C2', 'A1', 'A1', 'F2', 'D2', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'G1', 'G1', 'G2', 'F2', 'E2', 'E-2', 'D2', 'C#2', 'C2', 'B1', 'B-1', 'G1', 'G1', 'F2', 'D2', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'A1', 'A1', 'G1', 'A1', 'F1', 'D1', 'E4', 'D1', 'D1', 'D2', 'D1', 'E4', 'D1', 'E4', 'D1', 'D2', 'C2', 'B1', '4.10', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'C2', 'C2', 'C2', 'B1', 'B1', 'F1', 'D1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'C2', 'C2', 'B1', 'A1', 'A1', 'F1', 'D1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'G1', 'G1', 'G1', 'G1', 'G1', 'G1', 'F1', 'D1', 'D2', 'D1', 'D2', 'A1', 'C2', 'A1', 'A1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D2', 'D1', 'D2', 'A1', 'F1', 'G1', 'G1', 'F1', 'D1', 'D2', 'D1', 'D2', 'A1', 'C2', 'A1', 'A1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'B1', 'A1', 'F1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'B1', 'A1', 'A1', 'E2', 'G2', 'A2', 'G2', 'F#2', 'F2', 'E2', 'E-2', 'D2', 'C#2', 'C2', 'A1', 'A1', 'F2', 'D2', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'G1', 'G1', 'G2', 'F2', 'E2', 'E-2', 'D2', 'C#2', 'C2', 'B1', 'B-1', 'G1', 'G1', 'F2', 'D2', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'A1', 'A1', 'G1', 'A1', 'F1', 'D1', 'D1', 'D1', 'D2', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'C2', 'C2', 'C2', 'B1', 'B1', 'F1', 'D1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'C2', 'C2', 'B1', 'A1', 'A1', 'F1', 'D1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'G1', 'G1', 'G1', 'G1', 'G1', 'G1', 'F1', 'D1', 'D2', 'D1', 'D2', 'A1', 'C2', 'A1', 'A1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D2', 'D2', 'D1', 'G1', 'G1', 'F1', 'D1', 'D2', 'D1', 'D2', 'A1', 'C2', 'A1', 'A1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'B1', 'A1', 'F1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'C2', 'B1', 'A1', 'F1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D2', 'D1', 'D2', 'D1', 'D1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D1', 'D2', 'C2', 'B1', 'B-1', 'A1', 'G#1', 'G1', 'F#1', 'F1', 'D1', 'D1', 'C2', 'A1', 'G1', 'D1', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', '2.7', 'G3', '2.7', '2.7', 'G2', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', '2.5', 'G3', '2.5.7', '2.5.7', 'F2', 'D6', '9', '9', 'D7', 'D6', '5', '5', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '5', '5', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '5', '5', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '5', '5', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', '2.3', 'G3', '2.3.7', '2.3.7', 'E-2', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', 'E-3', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'B-2', 'D7', 'D5', 'D6', 'D6', 'D5', 'D6', '9', '9', 'D7', 'D6', '7', '7', 'G2', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', 'D7', 'D5', 'D6', 'D6', 'D5', '9.2', '9.2', '9.2', 'D2', 'D6', '9', '9', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', '6', '6', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', '9.2', '9.2', '9.2', 'D2', 'D7', 'D5', 'D6', 'D6', 'D5', '9.2', '9.2', '9.2', 'D2', 'D6', '9', '9', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', '6', '6', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', '9.2', '9.2', '9.2', 'D2', 'D7', 'D5', 'D6', 'D6', 'D5', '9.2', '9.2', '9.2', 'D2', 'D6', '9', '9', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', '6', '6', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', 'D4', 'D5', 'D5', 'D4', '9.2', '9.2', '9.2', 'D2', 'D7', 'D5', 'D6', 'D6', 'D5', '9.2', '9.2', '9.2', 'D2', 'D6', '9', '9', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', '6', '6', '9.2', '9.2', '9.2', 'D2', 'D7', 'D6', '2', '2', 'D7', '7.0', 'C4', '7.0', '7.0', 'C2', 'C4', 'E4', 'F#4', 'G4', 'G4', '9.2', 'D4', '9.2', '9.2', 'D2', 'D4', 'D4', 'E4', 'F#4', 'F#4', '7.11', '7.11', '7.11.2', 'G3', '7.11.2', 'G3', 'D3', '7.11.2', 'G3', '7.11.2', '7.11.2', '2.6', '2.6', '11.2.6', 'F#3', 'D4', '11.2.6', 'D4', '11.2.6', 'F#3', 'D4', 'D4', '11.2.6', 'B3', '11.2.6', '11.4', '11.4', '4.7.11', 'E3', 'E4', '4.7.11', 'B2', '4.7.11', 'E2', '4.7.11', '4.7.11', '11.2', '11.2', '4.7.11', 'E3', '4.7.11', 'B2', '2.6.9', 'E3', '2.6.9', '2.6.9', '7.0', '7.0', '0.4.7', '0.4', 'C2', '0.4', 'C2', '0.4.7', 'C2', 'E4', 'F#4', '0.4.7', 'C2', 'G4', '0.4.7', 'A1', '0.4.7', 'C2', 'G4', '0.4.7', '0.4.7', 'C2', '2.7', '2.7', '0.4.7', 'G1', '0.4.7', 'G1', '0.4.7', '2.6', '2.6', '9.2', '2.6.9', 'D2', '9.2', 'D2', '2.6.9', 'D2', 'D4', 'E4', '2.6.9', 'D2', 'F#4', '2.6.9', 'A1', '9.2', 'D2', '9.2', 'D4', '2.6.9', 'D2', '0.4', '0.4', '2.6.9', 'D2', '9.2', 'F#2', '11.2', '11.2', '9.2', '7.11.2', '7.11.2', '7.11', '7.11.2', 'G2', '7.11', 'D2', '7.11.2', 'G2', '7.11.2', 'B3', '7.11.2', '11.2.6', '11.2.6', '6.11', 'B1', 'D4', '6.11', '11.2.6', 'B1', 'D4', '11.2.6', 'B1', 'D4', 'D4', '6.11', 'B1', 'D4', '6.11', 'B1', 'B3', '4.8.11', '4.8.11', '11.4', '4.8.11', 'E1', 'E4', '11.4', 'E2', '4.8.11', 'E1', '2.8', '2.8', '4.8.11', 'E1', '1.6', '1.6', '4.8.11', '11.4', '11.4', '11.4', 'E1', '11.4', '11.4', '11.4', '4.8.11', 'E1', '1.6', '1.6', '11.4', '11.4', '4.8.11', 'E1', '11.4', '11.4', '2.7', '7.11', 'G2', 'G2', 'G2', 'B3', '2.7', '7.11', 'G2', '7.11.2', 'G1', 'C4', '4.9', '7.11.2', 'G1', 'C4', '4.9', '7.11.2', 'D2', 'D4', '7.11', '7.11.2', 'G2', '7.11.2', '7.11.2', 'D2', '7.11.2', 'G2', 'D2', '7.11.2', 'B1', '7.11.2', 'D2', 'G3', 'C3', '0.4.7', 'C3', 'C3', 'C2', 'C4', '4.7', 'C3', 'C2', 'G1', '0.4.7', 'C2', 'D4', '6.9', '0.4.7', 'C2', 'D4', '6.9', '0.4.7', 'G1', 'E4', '7.11', '0.4', 'C2', '0.4', '0.4.7', 'C2', '7.11.2', 'G2', 'G2', 'G1', 'E4', '7.11', 'G1', '7.11', 'G1', '4.9', '9.0.4', 'A2', 'A2', 'A2', 'C4', '4.7', '4.9', 'A2', 'E2', '9.0.4', 'A1', 'D4', '6.9', '9.0.4', 'C2', 'D4', '6.9', '9.0.4', 'E2', 'E4', '7.11', '4.9', 'A2', '4.9', '9.0.4', 'A2', 'E2', '7.11.2', 'G2', 'G2', 'A2', 'E4', '7.11', 'E2', '7.11.2', 'A1', 'E4', '7.11', '7.11.2', '2.6.9', 'D3', 'D3', 'D2', 'D4', '6.9', '7.0', 'C4', '7.0', '7.0', 'C2', 'C4', 'C4', 'E4', 'F#4', 'G4', 'G4', '9.2', 'D4', '9.2', '9.2', 'D2', 'F#4', 'E4', 'D4', 'F#4', 'F#5', 'F#5', 'E4', 'G5', 'G5', 'D4', 'A5', 'A5', 'B3', 'E4', '7.11', '7.11', '7.11.2', 'G2', 'G2', 'G3', '7.11.2', 'G3', 'D4', 'D3', '7.11.2', 'G3', 'B3', '7.11.2', '7.11.2', '6.9', '6.9', '11.2.6', 'F#2', 'F#2', 'F#3', 'E4', '11.2.6', 'D4', '11.2.6', 'F#3', 'B3', '11.2.6', 'A3', '11.2.6', '4.7', '4.7', '4.7.11', 'E2', 'E2', 'E3', 'G3', '4.7.11', 'B2', '4.7.11', 'E2', '4.7.11', '4.7.11', '11.4', '11.4', '4.7.11', 'E3', 'E3', 'E3', '4.7.11', 'B2', '2.6', '2.6', '2.6.9', 'D3', 'D3', 'E3', '2.6.9', '4.7', '4.7', '2.6.9', '4.7', '4.7', '0.4.7', '0.4', 'C3', 'C3', 'C2', 'E4', '0.4', 'C2', 'E4', '0.4.7', 'C2', 'E4', 'F#4', '0.4.7', 'C2', 'G4', '0.4.7', 'A1', '0.4.7', 'C2', 'G4', '0.4.7', '0.4.7', 'C2', '11.2', '11.2', '0.4.7', 'A2', 'A2', 'G1', '0.4.7', 'G1', '0.4.7', '9.2', '9.2', '9.2', '2.6.9', 'D3', 'D3', 'D2', '9.2', 'D2', 'D4', '2.6.9', 'D2', 'D4', 'E4', '2.6.9', 'D2', 'F#4', '2.6.9', 'A1', '9.2', 'D2', 'F#4', '9.2', 'D2', '2.6.9', 'D2', 'D2', 'D5', 'G5', 'D5', 'G5', '2.6.9', 'G2', 'G2', 'D2', 'E4', 'D2', 'F#5', 'F#5', '9.2', 'F#2', 'F#2', 'F#1', 'D4', '9.2', 'F#1', '7.11', '7.11', '7.11', '7.11.2', 'G2', 'G2', 'G2', 'E4', '7.11', 'D2', 'G4', '7.11.2', 'G2', 'E4', '7.11.2', 'D4', '7.11.2', '6.11', '6.11', '6.11', 'F#2', 'F#2', 'B1', 'B3', '6.11', '11.2.6', 'B1', '11.2.6', 'B1', '6.11', 'B1', 'A3', '6.11', 'B1', '4.8', '4.8', '11.4', '4.8.11', 'E2', 'E2', 'E1', 'B3', '11.4', 'E2', 'E4', '4.8.11', 'E1', '4.8.11', 'E1', '4.8.11', '11.4', 'E1', '11.4', '11.4', 'E1', '11.4', 'E1', '11.4', '11.4', '2.7', '7.11', '2.7', '2.7', 'G2', 'B3', '2.7', '7.11', 'G2', '7.11.2', '2.7', '2.7', 'G1', 'C4', '4.9', '7.11.2', 'G1', 'C4', '4.9', '7.11.2', 'D2', 'D4', '7.11', '7.11.2', '2.7', '2.7', 'G2', '7.11.2', '7.11.2', 'D2', '7.11.2', '2.7', '2.7', 'G2', 'D2', '7.11.2', 'B1', 'G3', '7.11.2', 'D2', 'C3', '0.4.7', '7.0', '7.0', 'C2', 'C4', '4.7', 'C3', 'C2', 'G1', '0.4.7', '7.0', '7.0', 'C2', 'D4', '6.9', '0.4.7', 'C2', 'D4', '6.9', '0.4.7', 'G1', 'E4', '7.11', '0.4', '7.0', '7.0', 'C2', '0.4', '0.4.7', 'C2', '11.2.4.7', '2.7', '2.7', 'G1', '7.11', 'G1', 'C4', '7.11', 'G1', 'C4', '4.9', '9.0.4', '4.9', '4.9', 'A2', 'C4', '4.7', '4.9', 'A2', 'E2', '9.0.4', '4.9', '4.9', 'A1', 'D4', '6.9', '9.0.4', 'C2', 'D4', '6.9', '9.0.4', 'E2', 'E4', '7.11', '4.9', 'A2', '4.9', '9.0.4', 'A2', 'E2', '7.11.2', '7.11.2', '7.11.2', 'A2', '7.11', 'E2', '7.11.2', 'A1', 'E4', '7.11', '7.11.2', 'E4', '2.6.9', '2.6.9', '2.6.9', 'D2', 'D6', '6.9', 'D7', 'D6', 'D7', 'D4', 'D6', 'D7', 'A3', 'D6', 'D7', 'D6', 'D7', 'D6', 'D7', 'D3', 'D6', 'D7', 'D6', 'D7', 'D6', 'D7', 'A3', 'D6', 'D7', 'D6', 'D7', 'D6', 'D7', 'D5', 'D6', 'G2', 'D6', 'A4', 'A5', 'D7', 'D6', 'G4', 'G5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'D6', 'A4', 'A5', 'D7', 'D6', 'G4', 'G5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'D6', 'A4', 'A5', 'D7', 'D6', 'G4', 'G5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'D6', 'A4', 'A5', 'D7', 'D6', 'G4', 'G5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'F2', 'D6', 'A4', 'A5', 'D7', 'D6', 'F4', 'F5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'D6', 'A4', 'A5', 'D7', 'D6', 'F4', 'F5', 'D7', 'D6', 'D4', 'D5', 'D7', 'D5', 'D6', 'D6', 'A4', 'A5', '9.2', 'D6', 'F4', 'F5', '9.2', 'D6', 'D4', 'D5', '9.2', 'D5', 'D6', 'D6', 'A4', 'A5', '9.2', 'D6', 'F4', 'F5', '9.2', 'D6', 'D4', 'D5', '9.2', 'D5', 'D6', 'E-2', 'D6', 'A4', 'A5', '9.2', 'D6', 'G4', 'G5', '9.2', 'D6', 'D4', 'D5', '9.2', 'D5', 'D6', 'D6', 'A4', 'A5', '9.2', 'D6', 'G4', 'G5', '9.2', 'D6', 'D4', 'D5', '9.2', 'D5', 'D6', 'E-3', 'D6', 'A4', 'A5', '9.2', 'D6', 'G4', 'G5', '9.2', 'D6', 'D4', 'D5', 'B-2', '9.2', 'D5', 'D6', 'D6', 'A4', 'A5', '9.2', 'D6', 'G4', 'G5', 'G2', '9.2', 'D6', 'D4', 'D5', '9.2', 'D5', 'D6', '9.2', '9.2', 'D2', 'D6', 'A4', 'A5', '9.2', '9.2', 'D2', '9.2', 'D6', 'F#4', 'F#5', '9.2', '9.2', 'D2', '9.2', 'D6', 'D4', 'D5', '9.2', '9.2', 'D2', '9.2', 'D5', 'D6', '9.2', '9.2', 'D2', 'D6', 'A4', 'A5', '9.2', '9.2', 'D2', '9.2', 'D6', 'F#4', 'F#5', '9.2', '9.2', 'D2', '9.2', 'D6', 'D4', 'D5', '9.2', '9.2', 'D2', '9.2', 'D5', 'D6', '9.2', '9.2', 'D2', 'D6', 'A4', 'A5', '9.2', '9.2', 'D2', '9.2', 'D6', 'F#4', 'F#5', '9.2', '9.2', 'D2', '9.2', 'F4', 'D6', 'D4', 'D5', '9.2', '9.2', 'D2', '9.2', 'F4', 'D5', 'D6', '9.2', '9.2', 'D2', 'D6', 'F4', 'A4', 'A5', '9.2', '9.2', 'D2', '9.2', 'D6', 'F#4', 'F#5', '9.2', '9.2', 'D2', '9.2', 'F4', 'D6', 'D4', 'D5', '9.2', '9.2', '9.2', '0.5', '0.5', 'F2', 'C6', 'C5', 'C6', 'C5', 'C6', 'C5', 'C6', '5.9.0', 'C5', '5.9.0', 'C6', 'C5', '5.9.0', 'C6', 'C5', '5.9.0', 'C6', '5.9.0', 'C5', '3.7.10', 'C6', 'C5', 'C6', 'C5', '3.7.10', 'C6', 'C5', '3.7.10', 'C6', 'C5', 'C6', 'C5', '3.7.10', 'C6', '3.7.10', 'C5', '2.5.9', 'C6', 'C5', 'C6', 'C5', '2.5.9', 'C6', 'C5', '2.5.9', 'C6', 'C5', 'C6', 'C5', '2.5.9', 'C6', '2.5.9', 'C5', '0.3.7', 'C6', 'C5', 'C6', 'E3', 'C5', 'F4', '0.3.7', 'F3', 'C6', 'A4', 'E3', 'C5', '0.3.7', 'C6', 'F3', 'C5', 'A4', 'C6', 'F3', 'C5', '0.3.7', 'F2', 'G4', 'F4', '5.9.0', '5.9.0', '5.9.0', '5.9.0', '5.9.0', '3.7.10', 'F2', '3.7.10', 'E-2', '3.7.10', 'C2', '3.7.10', 'F1', '3.7.10', '2.5.9', '2.5.9', '2.5.9', '2.5.9', '2.5.9', '0.3.7', 'F4', 'B-3', '0.3.7', 'C4', 'B-3', '0.3.7', 'C4', 'C4', '0.3.7', 'F3', 'C4', '5.9.0', '5.9.0', '5.9.0', '5.9.0', '5.9.0', '3.7.10', '3.7.10', '3.7.10', '3.7.10', '3.7.10', '2.5.9', 'F2', '2.5.9', 'E-2', '2.5.9', 'C2', '2.5.9', 'F1', '2.5.9', '0.3.7', 'F4', 'F4', '0.3.7', 'F4', 'F4', '0.3.7', 'F4', 'F4', '0.3.7', '3.8', '3.8', 'G#1', 'E-4', 'C5', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'E-5', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '3.8', '3.8', 'G#1', 'G#2', 'G#2', 'G#1', 'G#2', 'G#2', 'G#1', '8.0.3', '8.0.3', 'G#1', '8.0.3', '8.0.3', 'G#1', 'E2', 'E2', 'E2', 'E2', 'E1', '4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E2', 'E2', 'E1', 'E4', '11.4', '11.4', 'E2', 'E3', 'E3', 'E2', 'E2', 'E1', 'A4', 'E2', 'E2', 'E1', 'G#4', 'E4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E4', '9.2', '9.2', 'D2', 'D4', '6.11', '6.11', 'B1', 'E2', 'E2', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', '4.9', 'A1', 'E2', 'E2', 'E2', 'E2', 'E1', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'G#4', 'E1', '11.4', '11.4', 'E2', 'G#4', 'E3', 'E3', 'E2', 'E2', 'E1', 'G#4', 'F#4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E4', 'E1', 'F#4', '11.4', '11.4', 'E2', 'G#4', 'E2', 'E2', 'E2', 'E2', 'E1', 'E2', 'E2', 'E1', 'E4', '11.4', '11.4', 'E2', 'E1', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', '4.9', 'A1', 'E2', 'E2', 'E2', 'E2', 'E1', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'G#4', 'E1', '11.4', '11.4', 'E2', 'G#4', 'E2', 'E2', 'E1', 'G#4', 'F#4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E4', 'E1', '11.4', '11.4', 'E2', 'E4', 'E2', 'E2', 'E2', 'E2', 'E1', 'E4', 'E2', 'E2', 'E1', 'A2', 'A2', 'A2', 'A2', 'A1', 'B-2', 'B-2', 'B-2', 'B-2', 'B-1', 'B2', 'B2', 'B2', 'B2', 'B1', 'D3', 'D3', 'D3', 'D3', 'D2', 'E3', 'E3', 'E3', 'E3', 'E2', 'B2', 'B2', 'B2', 'B2', 'B1', 'B-2', 'B-2', 'B-2', 'B-2', 'B-1', 'A2', 'A2', 'A2', 'A2', 'A1', 'E2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', '4.9', 'A1', 'E2', 'E2', 'E2', 'E2', 'E1', 'G#4', 'E2', 'E2', 'E1', 'G#4', '11.4', '11.4', 'E2', 'F#4', 'E1', 'E4', '11.4', '11.4', 'E2', 'E4', 'E3', 'E3', 'E2', 'E2', 'E1', 'E2', 'E2', 'E1', 'C#4', 'E4', '11.4', '11.4', 'E2', 'E1', '11.4', '11.4', 'E2', 'F#4', 'E2', 'E2', 'E2', 'E2', 'E1', '8.1', '11.4', 'E2', 'E2', 'E1', '11.4', '11.4', 'E2', 'E1', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'E2', 'E1', 'E4', '4.9', '4.9', '4.9', '4.9', 'A1', 'E4', 'E2', 'E2', 'E1', 'G#4', 'E2', 'E2', 'E1', 'F#4', 'E2', 'E2', 'E4', '11.4', '11.4', 'E2', 'E2', 'E2', 'E1', '11.4', 'E2', '11.4', 'E2', 'E2', 'C#4', 'E2', 'E2', 'E4', '11.4', '11.4', 'E2', '11.4', '11.4', 'E2', '11.4', '11.4', 'E2', '11.4', '11.4', 'E2', 'E4', '11.4', '11.4', 'E2', 'E4', '11.4', '11.4', 'E2', '11.4', '11.4', 'E2', 'E4', '11.4', '11.4', 'E2', '9.2', '9.2', 'D2', 'D4', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'B3', 'D2', 'G4', 'D2', 'D2', 'D2', 'E4', 'D2', 'D2', 'A3', 'D2', 'G3', 'E2', '11.4', '11.4', 'E1', 'E2', 'E1', 'E2', '1.6', '1.6', 'E1', 'B2', '2.7', '2.7', 'E2', 'E2', 'E2', '11.4', '11.4', 'E1', 'B2', 'A2', 'E1', 'G2', '1.6', '1.6', 'E1', 'E2', '2.7', '2.7', 'E2', 'B2', '11.4', '11.4', 'E1', 'D3', 'E3', 'E1', 'F#3', '1.6', '1.6', 'E1', 'G3', '2.7', '2.7', 'E2', 'E2', 'E2', 'B3', 'E2', 'E2', 'E1', 'A3', '9.2', '9.2', 'D2', 'G3', 'A3', 'G3', 'E2', 'E2', 'E1', 'A3', '4.9', '4.9', 'A1', 'B3', 'E4', 'G4', '11.4', '11.4', 'E1', 'E1', '1.6', '1.6', 'E1', '2.7', '2.7', 'E2', 'E4', 'D4', 'B3', 'D4', 'E4', 'E4', 'B3', '11.4', '11.4', 'E1', 'D4', 'B3', 'D4', 'E1', 'B3', 'D4', 'B3', '1.6', '1.6', 'E1', 'D4', 'B3', 'E4', 'B3', '2.7', '2.7', 'E2', 'D4', 'D4', 'B3', 'B3', 'D4', 'D4', 'B3', 'B3', 'D4', 'B3', 'D4', 'B3', 'D4', 'B3', 'D4', 'B3', '11.4', '11.4', 'E1', 'D4', 'D4', 'B3', 'B3', 'D4', 'D4', 'E1', 'B3', 'B3', 'D4', 'D4', 'B3', '1.6', '1.6', 'E1', 'E4', 'D4', 'B3', '2.7', '2.7', 'E2', 'D4', 'B3', 'D4', 'E2', 'B3', 'D4', 'E2', 'E4', 'E2', 'E2', 'E1', '9.2', '9.2', 'D2', 'D5', 'C5', 'B4', 'A4', 'B4', 'A4', 'E2', 'E2', 'E1', 'B4', '4.9', '4.9', 'A1', 'A4', 'D5', 'G4', 'A4', 'C5', 'B4', 'D5', 'B4', 'A4', 'E5', 'B4', 'A4', 'E2', 'E2', 'E2', 'B4', 'A4', 'G4', 'E2', 'E2', 'E2', 'A4', 'B4', 'D5', 'E2', 'E2', 'E5', 'E5', '11.4', '11.4', 'E2', 'B4', 'D2', 'A4', 'B1', 'G4', 'A4', 'G4', 'E5', 'E2', 'E2', 'E2', 'E4', 'B4', 'G4', 'A4', 'E2', 'E2', 'E2', 'E4', 'D4', 'C4', 'G4', 'E2', 'E2', 'D4', 'B3', 'A4', 'G4', '11.4', '11.4', 'E2', 'A3', 'E4', 'G3', 'G4', 'E2', 'E4', 'E3', 'D4', 'C4', 'E-3', 'D4', 'B3', 'E-3', 'E-3', 'E1', 'E3', 'A3', 'E3', 'E3', 'E1', 'B2', 'G3', 'B2', 'B2', 'E1', 'E-3', 'E3', 'E3', 'E3', 'E3', 'E2', 'G3', 'E2', 'E3', 'E2', 'G3', 'E2', 'E2', 'E2', 'E2', 'E1', '7.11', '9.2', '9.2', 'D2', '7.11', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', 'A1', '4.9', 'E2', '3.4', '3.4', 'E1', '11.4', 'E3', 'E3', 'B2', 'B2', 'E2', 'B1', 'E3', 'E2', 'E3', 'E2', 'E2', 'D2', 'B1', 'D2', '3.4', '3.4', 'E1', 'E3', 'E3', 'B2', 'B2', 'E2', 'B1', 'B3', 'E3', 'E3', 'E2', 'A3', 'D3', 'D3', 'D2', 'G3', 'B2', 'B2', 'B1', 'E3', 'D3', 'E3', 'E3', 'D2', 'E3', 'D3', 'E-3', 'E-3', 'E1', 'B2', 'E3', 'E3', 'A2', 'G2', 'B2', 'B2', 'E2', 'B2', 'E3', 'B1', 'E3', 'E3', 'E3', 'E2', 'E3', 'D3', 'D3', 'E2', 'E3', 'B2', 'B2', 'E2', 'D4', 'E3', 'E3', 'E2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', 'D2', '9.2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', 'C#2', '4.9', 'E2', 'E2', '4', 'E1', '4', 'E1', 'F#2', 'F#2', 'E1', 'G2', 'G2', '4', 'E2', '4', 'E2', 'E2', 'E2', 'E2', '4', 'E1', '4', 'E1', 'F#2', 'F#2', 'F#2', 'G2', 'G2', '4', 'G2', '4', 'G2', 'G2', 'E2', 'E2', '4', 'E2', '4', 'E2', 'F#2', 'F#2', 'F#2', 'G2', 'G2', '4', 'G2', '4', 'G2', 'G2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', 'A1', 'E2', 'E2', 'E2', 'E1', '11.4', 'E1', 'F#2', 'F#2', 'F#1', 'G2', 'G2', '11.4', 'E2', '11.4', 'E2', '11.4', 'E2', 'E2', 'E2', 'E2', 'E1', '11.4', 'E1', 'F#2', 'F#2', 'F#1', 'G2', 'G2', '11.4', 'E2', '11.4', 'E2', '11.4', 'E2', 'E2', 'E2', 'E2', 'E1', '11.4', 'E1', 'F#2', 'F#2', 'F#1', 'G2', 'G2', '11.4', 'G1', '11.4', 'G1', '11.4', 'G1', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'E1', '4.9', '4.9', '4.9', 'A1', '11.4', '11.4', '11.4', 'E1', '9.2', '9.2', '9.2', 'D2', 'C3', '0', '7.0', '7.0', 'C2', '11', '11.4', '2', '11.4', '11.4', 'E1', 'E1', 'E1', 'E2', 'D2', 'B1', 'D2', 'E2', 'E2', 'E2', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', '11.4', '11.4', 'E2', 'E2', 'E2', 'E2', '11.4', '11.4', 'D2', 'E2', 'E2', 'B1', '11.4', '11.4', 'D2', '4', '4', 'E2', 'E2', 'E2', 'E2', 'E2', 'D2', 'B1', 'D2', 'E2', 'E2', 'E2', 'E1', '9.2', '9.2', '9.2', 'D2', 'E2', 'E2', 'E2', 'A1', '4.9', '4.9', '4.9', 'E4', 'E2', '9.2', '9.2', 'E2', 'E2', 'G4', '11.4', '11.4', 'E2', 'E4', 'G4', '11.4', '11.4', 'E2', 'A4', 'E4', 'F#2', 'F#2', 'E2', 'G4', 'E4', 'G2', '11.4', '11.4', 'G2', 'E2', 'A4', 'G4', '9.2', '9.2', 'D2', 'B4', 'E4', 'A4', 'G4', '6.11', '6.11', 'B1', 'B4', 'A4', 'G4', 'E4', '9.2', '9.2', 'D2', 'B4', 'G4', 'E2', '9.2', 'E2', '9.2', 'E2', 'D5', 'A4', '11.4', '11.4', 'F#2', 'B4', 'B4', 'D5', 'A4', '11.4', '11.4', 'G2', 'E-5', 'B4', 'E5', 'G4', 'F#2', 'F#2', 'A2', 'B4', 'B4', 'G2', '11.4', 'G2', '11.4', 'B2', 'D5', 'D5', '9.2', '9.2', 'B2', 'E5', 'B4', 'D5', 'D5', '6.11', '6.11', 'C3', 'B4', 'E-5', 'D5', 'E5', '9.2', '9.2', 'D3', 'D5', 'B4', 'E2', '9.2', 'E2', '9.2', 'E3', 'B4', 'D5', '11.4', '11.4', 'E3', 'E5', 'D5', 'D5', '11.4', '11.4', 'E3', 'B4', 'E5', 'D5', 'F#2', 'F#2', 'E3', 'D5', 'G2', '11.4', 'G2', '11.4', 'E3', 'E5', 'B4', '9.2', '9.2', 'E3', 'F#5', 'D5', '6.11', '6.11', 'E3', 'E5', '9.2', '9.2', 'E3', 'E2', 'E2', 'B2', 'E5', '9.2', '9.2', 'D3', 'F#5', 'B2', 'A2', 'E5', 'E2', 'E2', 'A2', 'D5', '4.9', '4.9', 'G2', 'B4', 'F#2', 'D5', 'E2', 'E5', 'E5', '9.2', 'E2', '9.2', 'E2', 'D2', 'D5', 'D5', '11.4', '11.4', 'E2', 'B4', 'B4', '11.4', '11.4', 'E2', 'B4', 'A4', 'D5', 'F#2', 'F#2', 'E2', 'G4', 'E5', '11.4', 'G2', '11.4', 'G2', 'E2', 'E4', 'D5', '9.2', '9.2', 'D2', 'E5', 'A5', 'B4', '6.11', '6.11', 'B1', 'B4', 'A4', '9.2', '9.2', 'D2', 'G4', '2.7', 'E2', '2.7', 'E2', 'E2', 'E4', '4.9', '4.9', 'D2', 'E5', 'A5', '4.9', '4.9', 'B1', 'F#2', 'F#2', 'A1', 'E5', '4.9', 'G2', '4.9', 'G2', 'G1', 'G5', '2.7', '2.7', 'F#1', 'D5', '11.4', '11.4', 'E1', 'E5', '2.7', '2.7', 'F#1', 'E5', '11.4', 'E2', '11.4', 'E2', 'E1', 'G5', '2.7', '2.7', 'E1', 'D5', '11.4', '11.4', 'E1', 'E5', 'F#2', 'F#2', 'D5', '4.7.11', '4.7.11', 'E2', 'B4', 'E2', 'A4', 'E2', 'B-4', 'E2', 'B4', 'D5', 'E2', 'E2', 'E1', 'B4', 'B4', '9.2', '9.2', 'D2', 'B4', 'A4', 'D2', 'A4', 'B-4', 'D2', 'G4', 'B4', 'E2', 'E2', 'C#2', 'A4', 'B4', '4.9', '4.9', 'B1', 'B4', 'B4', 'C#2', 'A4', 'A4', 'D2', 'B5', 'G4', 'E2', '4', 'D2', 'A4', 'E2', 'B4', '4', 'E2', 'A4', 'F#2', 'E2', 'B5', 'B5', 'G2', '4', 'E2', 'D2', 'A5', '4', 'B1', 'G5', '4', 'D2', 'A5', 'B5', 'E2', '4', 'D2', 'E5', 'E2', 'G5', 'A5', '4', 'E2', 'E5', 'G5', 'F#2', 'E2', 'E5', 'A5', 'G2', '4', 'E2', 'E5', 'D2', 'G5', '4', 'B1', 'E5', '4', 'D2', 'G#5', 'E5', 'E2', '4', 'E1', '4', 'E1', 'B4', 'F#2', 'E1', 'D5', 'G#5', 'G2', '4', 'D2', 'E5', 'E2', 'G5', '4', 'E2', 'B4', '4', 'E2', 'D5', 'D5', 'E2', 'E2', 'E1', 'E5', 'E5', '9.2', '9.2', 'D2', 'G5', 'D5', '4.9', '4.9', 'A1', 'B3', 'E2', '4', 'E1', 'D4', '2.6', '2', 'D2', 'C4', 'C3', 'G3', 'C2', 'C3', 'G3', 'B3', 'D4', 'B3', 'D4', 'C4', 'E4', 'F#4', 'G4', 'G4', 'D4', 'D4', 'E4', 'F#4', 'F#4', 'E4', 'D4', 'B3', 'B5', 'G2', '7.11.2', 'G1', 'E4', '7.11.2', 'A5', '7.11.2', 'G1', 'D4', 'G5', '7.11.2', 'G1', 'D4', '7.11.2', 'G1', '7.11.2', 'G1', 'A5', 'D3', '11.2.6', 'D2', 'E4', '2.6.9', 'G5', '11.2.6', 'D2', 'D4', 'F#5', '11.2.6', 'D2', 'B3', 'C2', '11.2.6', 'B1', 'D4', '11.2.6', 'D2', '4.7', 'B2', '4.7.11', 'E2', 'E4', '4.7.11', '4.7.11', 'B1', '4.7.11', 'E1', '4.7.11', 'E2', '4.7.11', 'B1', 'B2', '4.7.11', 'E2', 'D3', 'D3', '4.7.11', 'B2', 'E3', '2.6.9', 'D2', 'D3', 'B2', '2.6.9', 'A2', 'G2', '2.6.9', 'C3', '0.4.7', '0.4', 'C2', '0.4.7', '0.4', 'C2', 'E4', 'G1', 'E4', '0.4.7', 'C2', 'E4', 'F#4', '0.4.7', 'C2', 'G4', '0.4.7', 'G1', '0.4.7', 'C2', 'G4', '0.4.7', '0.4.7', 'B2', '0.4.7', 'G1', '4.7.11', '0.4.7', '0.4.7', 'D3', '2.6.9', '9.2', 'D2', '2.6.9', '9.2', 'D2', 'D4', 'D4', '2.6.9', 'D2', 'D4', 'E4', '2.6.9', 'D2', 'F#4', '2.6.9', '9.2', 'D2', 'F#4', '9.2', '2.6.9', 'D2', 'C3', '2.6.9', 'C2', 'E4', '7.9.2', '9.2', 'C2', 'D4', '9.2', 'G2', '7.11', '7.11.2', 'G1', 'B3', '7.11.2', '7.11', 'G1', '7.11.2', 'G2', 'D2', 'A3', '7.11.2', 'G1', 'B3', '7.11.2', 'B2', '6.11', 'B1', 'E4', '2.6.9', '6.11', '11.2.6', 'B1', 'D4', '11.2.6', 'B1', 'D4', 'F#2', '6.11', 'B1', 'B3', '6.11', 'E2', '11.4', '4.8.11', 'E2', 'E4', '4.8.11', 'E2', '11.4', 'E2', 'E2', '4.8.11', 'E2', 'B1', '2.8', '4.8.11', 'E1', '1.6', '4.8.11', '11.2.4', '11.4', 'E1', '11.4', '11.2.4', '4.8.11', 'E1', '1.6', 'B1', '11.4', '4.8.11', 'E2', '11.4', '11.4', '7.11.2', 'G1', '7.11.2', '7.9.0', '7.11.2', '7.11.2', '0.4.7', '0.4.7', '0.4', 'C2', 'C3', '0.4.7', '0.4', 'C2', 'G4', 'G3', '7.11.2', '7.11', 'G2', 'G4', 'D3', '9.2', '2.6', 'D2', 'F#4', 'A2', 'D3', 'F#4', 'A2', 'D2', 'E3', '11.4', '4.7.11', 'E2', 'G4', 'E2', 'D2', 'E3', 'A4', 'B2', 'E3', '4.9', '2.4', 'A1', 'A4', 'E2', 'A4', 'A3', '4.9', '1.4', 'A2', 'A4', 'A2', 'G4', 'C3', '0.4.7', '0.4.7', '0.4', 'C2', 'E4', 'F#4', 'G4', 'G4', 'C3', 'C3', 'C2', 'B2', 'B2', 'B1', 'G2', 'G2', 'G1', 'A2', 'A2', 'A1', 'E4', 'G4', 'G4', 'G4', 'A4', 'B4', 'B4', 'C5', 'D5', 'D5', 'G2', '2.7', '7.11.2', 'G1', '7', '7.11.2', 'G1', 'G1', '7.11.2', 'G2', '9', '7.11.2', 'G2', '11', '7.11.2', 'D2', '11', 'G3', '7.11.2', 'G1', 'B4', '7.11.2', 'G4', 'G4', '7.11.2', 'G1', 'E4', 'D4', 'G1', 'D4', '7.11.2', 'G2', 'C#4', 'B3', 'B3', '7.11.2', 'G2', 'A3', 'G3', '7.11.2', 'D2', 'C3', '7.0', '0.4.7', 'C2', '7', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', '0.4.7', 'C2', '11', '0.4.7', 'C2', '11', 'D4', '0.4.7', 'C2', '0.4.7', '0.4.7', 'C2', 'C2', '2.7', '0.4.7', 'G1', '2.7', '0.4.7', 'G1', '2.7', '0.4.7', 'G1', 'D4', 'A2', '9.0.4', 'A1', '7', 'C4', 'B3', '9.0.4', 'A1', 'A1', '9.0.4', 'A1', '9', 'F4', '9.0.4', 'A1', '11', '9.0.4', 'E2', '11', '9.0.4', 'A2', '9.0.4', 'F4', '9.0.4', 'A2', 'D4', 'C4', 'A2', 'C4', '9.0.4', 'A2', '0.4.7', 'B3', 'G2', 'A3', 'C3', '9.0.4', 'E2', '0.4.7', 'B2', 'A2', '9.0.4', 'G2', '2.7', '7.11.2', 'G2', '7.11.2', '7.11.2', 'G2', 'G2', 'D3', '7.11.2', 'G2', 'E3', 'G3', 'E3', '7.11.2', 'G2', 'G3', 'B3', '7.11.2', 'G2', 'D4', '7.11.2', 'G2', 'B3', '7.11.2', 'D4', 'G4', '7.11.2', 'G2', 'A4', 'B4', 'G2', 'A4', '7.11.2', 'G2', 'G4', 'E2', 'E4', 'D4', '7.11.2', 'F2', 'B3', 'D4', '7.11.2', 'G2', 'E4', '2.7', '7.11.2', 'G1', '7', 'G4', '7.11.2', 'G1', '7.11.2', 'G1', '9', '7.11.2', 'G1', '11', '7.11.2', '11', '7.11.2', 'G2', '7.11.2', 'B4', '7.11.2', 'G2', '7.11.2', 'G2', '7.11.2', 'G2', 'A4', 'G4', '7.11.2', 'E2', 'E4', '7.0', '0.4.7', 'C2', '7', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', '0.4.7', 'C2', '11', '0.4.7', 'C2', '11', 'B4', '0.4.7', 'C2', '0.4.7', 'C5', '0.4.7', 'C2', 'C2', '2.7', '0.4.7', 'G2', 'G2', '0.4.7', 'G2', 'G4', 'A4', '0.4.7', 'G2', 'G4', '4.9', '9.0.4', 'A2', '7', 'A4', 'G4', '9.0.4', 'A2', 'A2', '9.0.4', 'A2', '9', 'E2', 'C5', '9.0.4', 'C2', '11', 'B4', '9.0.4', 'A2', '11', '9.0.4', 'A2', '9.0.4', 'G4', '9.0.4', 'A2', 'F4', 'D4', 'A2', 'F4', '9.0.4', 'C3', '0.4.7', 'G4', 'A2', 'D4', 'E4', '9.0.4', 'G2', '0.4.7', 'D4', 'C4', '9.0.4', 'A2', 'D4', 'G2', '7.11.2', 'G2', '7.11.2', 'G2', '7.11.2', 'G2', 'G2', 'D4', '7.11.2', 'G2', 'G4', 'F2', '7.11.2', 'D2', 'G4', '7.11.2', 'C3', 'G4', '7.11.2', 'B2', 'A4', '7.11.2', 'A2', '7.11.2', 'G2', 'A4', 'F2', 'B4', '7.11.2', 'E2', 'A4', 'D2', 'G4', '7.11.2', 'C2', '7.11.2', 'G2', 'D4', '2.7', '7.11.2', 'G1', '7', 'E4', 'D4', 'B3', '7.11.2', 'G1', 'G1', '7.11.2', 'G1', '9', '7.11.2', 'G1', '11', 'B3', '7.11.2', 'G1', '11', 'D4', '7.11.2', 'G2', 'G4', '7.11.2', '7.11.2', 'G2', 'A4', 'G2', '7.11.2', 'G2', 'G4', 'G2', '7.11.2', 'E2', 'B4', '7.11.2', 'D2', '7.0', '0.4.7', 'C2', '7', 'G4', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', '0.4.7', 'C2', '11', 'B4', '0.4.7', 'C2', '11', 'A4', '0.4.7', 'C2', 'G4', '0.4.7', 'E4', '0.4.7', 'C2', 'D4', 'C2', 'G4', 'A4', '2.7', '0.4.7', 'G2', 'B4', 'G2', '0.4.7', 'G2', '0.4.7', 'G2', 'G4', 'A4', '4.9', '9.0.4', 'A2', '7', 'G4', 'A2', '9.0.4', 'A2', 'A2', '9.0.4', 'A2', '9', 'G2', 'C5', '9.0.4', 'E2', '11', '9.0.4', 'C2', '11', '9.0.4', 'A1', '9.0.4', '9.0.4', 'A1', 'A1', 'C5', '9.0.4', 'A1', '0.4.7', 'B4', '9.0.4', 'A1', '0.4.7', 'A4', '9.0.4', 'G4', 'G2', '7.11.2', 'G2', '7.11.2', 'G2', 'F4', '7.11.2', 'G2', 'G4', 'G2', 'F4', '7.11.2', 'G2', 'G2', 'G4', '7.11.2', 'G2', 'F4', '7.11.2', 'G2', '7.11.2', 'G2', '7.11.2', 'C3', '7.11.2', 'B2', 'D5', 'G2', 'C5', '7.11.2', 'E2', 'B-4', 'D2', 'B4', '7.11.2', 'B1', 'G4', '7.11.2', 'A1', 'B4', '2.7', '7.11.2', 'G1', '7', 'G4', 'D4', '7.11.2', 'G1', 'G1', '7.11.2', 'G2', '9', '7.11.2', 'G2', '11', '7.11.2', 'D2', '11', '7.11.2', 'G1', 'B4', '7.11.2', '7.11.2', 'G1', 'C5', 'G1', '7.11.2', 'G2', 'B4', '7.11.2', 'G2', '7.11.2', 'D2', '7.0', '0.4.7', 'C2', '7', 'E5', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', '0.4.7', 'C2', '11', '0.4.7', 'C2', '11', 'E5', '0.4.7', 'C2', '0.4.7', 'D5', '0.4.7', 'C2', 'C5', 'C2', 'D5', '2.7', '0.4.7', 'G1', 'C5', 'G1', '0.4.7', 'B4', 'G1', '0.4.7', 'C5', '4.9', '9.0.4', 'A1', '7', 'B4', 'A4', '9.0.4', 'A1', 'A1', '9.0.4', 'A1', '9', '9.0.4', 'A1', '11', '9.0.4', 'E2', '11', '9.0.4', 'A2', '9.0.4', 'G5', '9.0.4', 'A2', 'A2', '9.0.4', 'A2', '4.7.9.0', 'G2', '9.0.4', 'E2', '4.7.9.0', '9.0.4', 'G2', '7.11.2', 'G2', '7.11.2', 'G5', '7.11.2', 'G2', 'G2', 'G5', '7.11.2', 'G2', 'D5', '7.11.2', 'G2', '7.11.2', 'G2', 'B4', '7.11.2', 'G2', 'C5', '7.11.2', 'B4', 'C5', '7.11.2', 'G2', 'B4', 'C5', 'G2', 'B4', '7.11.2', 'G2', 'C5', 'E2', 'B4', 'E5', '7.11.2', 'F2', 'D5', 'C5', '7.11.2', 'G2', 'B4', '2.7', '7.11.2', 'G1', '7', 'C5', 'G4', '7.11.2', 'G1', 'G1', '7.11.2', 'G2', '9', '7.11.2', 'G2', '11', '7.11.2', 'D2', '11', 'B-4', '7.11.2', 'G1', 'B4', '7.11.2', 'D5', '7.11.2', 'G1', 'B-4', 'G1', 'B4', '7.11.2', 'G2', 'G4', 'F4', '7.11.2', 'G2', 'G4', 'E4', 'D4', '7.11.2', 'D2', 'C4', '7.0', '0.4.7', 'C2', '7', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', 'G5', '0.4.7', 'C2', '11', '0.4.7', 'C2', '11', '0.4.7', 'C2', '0.4.7', 'G5', '0.4.7', 'C2', 'C2', '2.7', '0.4.7', 'G1', 'G1', '0.4.7', 'G1', '0.4.7', 'B5', '4.9', '9.0.4', 'A1', '7', '9.0.4', 'A1', 'A1', '9.0.4', 'A1', '9', '9.0.4', 'A1', '11', 'A5', '9.0.4', 'E2', '11', '9.0.4', 'A2', '9.0.4', '9.0.4', 'A2', 'A2', 'G5', '9.0.4', 'A2', '0.4.7', 'E5', 'G2', 'D5', '9.0.4', 'E2', '0.4.7', 'E5', '9.0.4', 'D5', 'G2', '7.11.2', 'G2', '7.11.2', '7.11.2', 'G2', 'G2', '7.11.2', 'G2', 'B4', '7.11.2', 'G2', 'C5', '7.11.2', 'G2', 'D5', '7.11.2', 'G2', '7.11.2', '7.11.2', 'G2', 'D5', 'G2', 'B4', 'A4', '7.11.2', 'G2', 'G4', 'E2', 'B4', '7.11.2', 'F2', 'A4', 'G4', 'D4', '7.11.2', 'G2', 'G4', 'E4', '2.7', '7.11.2', 'G1', '7', 'D4', 'C4', 'D4', '7.11.2', 'G1', 'C4', 'B3', 'A3', 'G1', 'C4', 'B3', '7.11.2', 'G2', '9', 'C4', 'D4', '7.11.2', 'G2', '11', '7.11.2', 'D2', '11', '7.11.2', 'G1', '7.11.2', 'C4', '7.11.2', 'G1', 'D4', 'G1', 'E4', '7.11.2', 'G2', 'G4', 'B4', '7.11.2', 'G2', 'G4', '7.11.2', 'D2', '7.0', '0.4.7', 'C2', '7', 'B4', '0.4.7', 'C2', 'C5', 'C2', 'D5', '0.4.7', 'C2', '9', 'B4', 'C5', '0.4.7', 'C2', '11', 'B4', '0.4.7', 'C2', '11', 'G4', '0.4.7', 'C2', 'E4', '0.4.7', 'D4', '0.4.7', 'C2', 'C4', 'C2', 'E4', '2.7', '0.4.7', 'G1', 'D4', 'G1', 'C4', '0.4.7', 'G1', 'B3', '0.4.7', 'A3', '4.9', '9.0.4', 'A1', '7', '9.0.4', 'A1', 'A1', '9.0.4', 'A1', '9', 'G4', '9.0.4', 'A1', '11', '9.0.4', 'E2', '11', '9.0.4', 'A2', '9.0.4', '9.0.4', 'A2', 'A2', '9.0.4', 'A2', '0.4.7', 'G2', '9.0.4', 'E2', '0.4.7', '9.0.4', 'D5', 'G2', '7.11.2', 'G2', '7.11.2', 'C5', '7.11.2', 'G2', 'D5', 'G2', '7.11.2', 'G2', 'D5', 'C5', '7.11.2', 'G2', 'D5', '7.11.2', 'G2', '7.11.2', 'G2', '7.11.2', 'D5', '7.11.2', 'G2', 'C5', 'G2', 'D5', '7.11.2', 'G2', 'E2', 'D5', '7.11.2', 'F2', 'C5', '7.11.2', 'G2', 'D5', '2.7', '7.11.2', 'G1', '7', '7.11.2', 'G1', 'G4', 'A4', 'G4', 'G1', 'D4', '7.11.2', 'G2', '9', '7.11.2', 'G2', '11', '7.11.2', 'D2', '11', '7.11.2', 'G1', '7.11.2', '7.11.2', 'G1', 'G1', '7.11.2', 'G2', '7.11.2', 'G2', '7.11.2', 'D2', '7.0', '0.4.7', 'C2', '7', '0.4.7', 'C2', 'C2', '0.4.7', 'C2', '9', '0.4.7', 'C2', '11', '0.4.7', 'C2', '11', '0.4.7', 'C2', '0.4.7', '0.4.7', 'C2', 'C2', '2.7', '0.4.7', 'G1', 'G1', '0.4.7', 'G1', '0.4.7', '7', '9', '11', '11', '0.4.7', '0.4.7', '7.11.2', '11.4', '11.4', '11.4', '11.4', '11.4', '4.8', '11.4', 'E3', '11.4', '11.4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', 'B4', 'E5', 'F#5', 'F#5', 'F#5', '11.4', 'G#5', '11.4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'B4', '11.4', 'E5', '4.8.11', 'F#5', '11.4', 'A5', '11.4', '11.4', 'G#5', '11.4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'B4', '11.4', 'E5', '4.8.11', 'F#5', '11.4', 'F#5', '11.4', 'A5', '11.4', '11.4', 'G#5', '11.4', '11.4', 'F#5', '11.4', 'A5', '4.8.11', 'G#5', '11.4', 'F#5', '11.4', '11.4', '11.4', 'A5', '11.4', '11.4', 'G#5', '11.4', 'F#5', 'A5', '11.4', 'G#5', '11.4', 'F#5', '9.11.4', 'F#5', '4.9', '4.9', '4.9', '4.9', 'G#5', '4.9', 'F#5', 'E5', 'C#5', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', 'F#5', '4.9', 'F#5', '4.9', 'E5', '4.9', 'C#5', '4.9', 'B4', '11.4', 'A4', '11.4', 'G#4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '4.8.11', '4.8', '11.4', '11.4', '3.6', '6.11', '6.11', '1.4', '11.3', '6.11', '6.11', '6.11', '6.11', '6.11', '6.11', 'A5', '4.9', '4.9', 'G#5', '4.9', 'A5', '4.9', 'G#5', '4.9', '4.9', 'E5', '4.9', 'F#5', '4.9', 'E5', '11.4', '11.4', 'E5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'B4', 'E5', 'F#5', '6.9', '6.9', '11.4', '8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '9.11.4', '11.4', '11.4', '11.4', '11.4', 'B4', '11.4', 'E5', '11.4', 'F#5', '11.4', 'F#5', '11.4', 'A5', '11.4', '11.4', 'G#5', '11.4', '11.4', 'A5', '11.4', 'G#5', '11.4', 'E5', '11.4', 'B4', '11.4', 'A5', '11.4', '11.4', 'G#5', '11.4', '11.4', 'A5', '11.4', 'G#5', '11.4', 'F#5', '11.4', 'F#5', '9.11.4', '2.6', '2.4.9', '2.4.9', '2.4.9', '2.4.9', '4.8', '2.4.9', '1.6', 'E5', 'A4', 'C#5', '2.4.9', 'E4', '2.4.9', '2.4.9', '4.9', '4.9', '4.9', '4.9', 'F#5', '4.9', 'F#5', 'E5', '4.9', 'C#5', '4.9', 'B4', '4.9', '11.4', 'A4', '11.4', 'G#4', '11.4', '11.4', '11.4', '4.8.11', 'G#4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '4.8.11', '4.8', '11.4', '11.4', '3.6', '6.11', '6.11', '1.4', '11.3', '6.11', '6.11', '6.11', '6.11', '6.11', '6.11', 'A5', '4.9', '4.9', 'G#5', '4.9', 'A5', '4.9', 'G#5', '4.9', '4.9', 'E5', '4.9', 'E5', '4.9', 'E5', '11.4', 'B4', '11.4', 'C#5', '11.4', 'E5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '9.1.2', '11.4', '8.11', '4.8.11', '1.2.4', '4.8.11', '11.2.4', '2.6.9', '4.8.11', '11.2.4', '6', '9.2', '4', '11.4', 'G#3', '6.8.11', 'A3', '4.9', 'E5', '11.4', 'A3', 'E-5', '6.11', 'B3', '11.3.6', '9.1.4', '6.11', '6.11', '6.11', 'E-5', '6.11', 'E-5', '6.11', '6.11', 'E5', 'A3', '9.1.4', '9.1.4', '9.11.4', '9.1.4', 'E-5', '9.11.4', 'C#5', '9.11.4', '9.11.4', '9.11.4', 'F#5', '9.11.4', '9.11.4', '9.11.4', '11.3.6', '6.11', '11.3.6', 'F#5', '6.11', 'F#5', '6.11', 'F#5', '6.11', 'F#5', '6.11', 'F#5', '6.11', 'F#5', 'F#5', '6.11', 'G#5', 'A3', '9.1.4', '9.1.4', 'F#5', '9.11.4', '9.1.4', 'A3', 'E5', '9.11.4', 'E5', '9.11.4', 'C#5', '9.11.4', 'E5', 'C#5', '9.11.4', '9.11.4', 'E5', '9.11.4', '9.1', 'E5', 'F#5', 'F#5', '9.11.4', 'G#5', '8.1', '8.11.1.4', '1.4.8', 'C#4', '8.1', 'F#5', '8.1', 'E5', '8.1', 'C#5', '8.1', '8.1', 'E5', '8.1', 'F#5', '8.1', 'G#5', '6.9.1', '4.6', 'C#4', 'B4', '4.6', 'B3', '6.9.1', 'C#6', '6.9.1', 'G#5', '6.9.1', 'B-4', 'F#5', '6.9.1', 'B-4', 'B-3', 'G#5', 'F#5', '6.9.1', 'F#4', 'E5', '6.9.1', 'C#4', '6.9.1', '6.10', '4.9', '9.1.4', '9.1.4', 'A5', '4.9', 'A5', '4.9', 'A5', '4.9', 'G#5', '6.11', '11.3.6', '11.3.6', 'F#5', '6.11', 'E5', 'E5', '8.1', '1.4.8', '1.4.8', '1', 'G#4', 'G#4', '8.1', 'C#5', '1.4.8', 'E5', '8.1', '1.4.8', '6.11', '11.3.6', '11.3.6', '4.9', '9.1.4', '9.1.4', 'E5', '4.9', 'E5', '4.9', 'E5', '4.9', 'E-5', '6.11', '11.3.6', '11.3.6', 'B4', 'C#5', '6.11', 'B4', 'B4', 'C#5', '8.1', '1.4.8', '1.4.8', 'B4', '1', 'G#4', 'G#4', 'G#4', '8.1', '1.4.8', '8.1', '1.4.8', '6.11', '11.3.6', '11.3.6', 'A5', '4.9', '9.1.4', '9.1.4', '4.9', 'A5', '4.9', 'B5', '4.9', 'G#5', '6.11', '11.3.6', '11.3.6', 'B4', 'F#5', '6.11', 'G#5', 'B4', 'F#5', '11.4', '11.4', '4.8.11', 'F#5', '11.4', '11.4', '4.8.11', '11.4', 'E5', '11.4', '9.11.4', '9.11.4', '4.8.11', 'B4', 'B3', 'B3', 'E5', 'E4', 'E4', 'F#5', 'F#4', 'F#4', 'G#5', 'G#4', 'G#4', '11.4', '11.4', 'G#5', 'G#4', 'G#4', '11.4', '11.4', '11.4', 'A5', 'A4', 'A4', 'G#5', 'G#4', 'G#4', '4.8.11', '6.7', '4.5', '6.7', '4.5', '6.7', '4.5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'B4', 'B3', 'B3', 'E5', 'E4', 'E4', '4.8.11', '11.4', 'F#5', 'F#4', 'F#4', '11.4', 'G#5', 'G#4', 'G#4', '11.4', 'G#5', 'G#4', 'G#4', '11.4', '11.4', 'A5', 'A4', 'A4', '11.4', 'G#5', 'G#4', 'G#4', '11.4', 'E5', 'E4', 'E4', 'B4', 'B3', 'B3', 'G#5', 'G#4', 'G#4', '4.8.11', 'A5', 'A4', 'A4', '11.4', 'G#5', 'G#4', 'G#4', 'E5', 'E4', 'E4', '11.4', 'B4', 'B3', 'B3', 'A4', 'A3', 'A3', '11.4', 'G#4', 'G#3', 'G#3', '11.4', '11.4', '6.7', '6.7', '6.7', '11.4', '11.4', '4.8.11', '11.4', 'C#6', 'C#5', 'C#5', '11.4', '4.9', 'D6', 'D5', 'D5', '4.9', 'C#6', 'C#5', 'C#5', '4.9', '4.9', 'A5', 'A4', 'A4', 'E5', 'E4', 'E4', '4.9', 'D5', 'D4', 'D4', 'C#5', 'C#4', 'C#4', 'D5', 'D4', 'D4', '4.9', 'C#5', 'C#4', 'C#4', 'A4', 'A3', 'A3', '4.9', 'E4', 'E3', 'E3', '4.9', 'E-4', '1.2', 'E-3', '1.2', 'E-3', '1.2', '4.9', '4.9', '4.9', '4.9', '4.9', 'C#6', 'C#5', 'C#5', '4.9', 'D6', 'D5', 'D5', 'C#6', 'C#5', 'C#5', 'D6', 'D5', 'D5', '4.9', 'C#6', 'C#5', 'C#5', 'A5', 'A4', 'A4', '4', 'E5', 'E4', 'E4', '11.4', 'G#5', 'A5', 'G#4', 'A4', 'G#4', 'A4', 'G#5', 'G#4', 'G#4', 'A5', 'A4', 'A4', '11.4', 'G#5', 'G#4', 'G#4', 'A5', 'A4', 'A4', '11.4', 'G#5', 'E5', 'G#4', 'E4', 'G#4', 'E4', '11.4', 'B4', 'B3', 'B3', '11.4', 'A4', 'A3', 'A3', 'G#4', 'G#3', 'G#3', '4.8.11', '11.4', '11.4', '6.7', '4.5', '6.7', '4.5', '6.7', '4.5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.3.6', '10.2.5', '8.9.0.1.3.4', '4.8.11', '6.7.10.11.1.2', '4.5.8.9.11.0', '2.3.6.7.9.10', '0.1.4.5.7.8', '11.4', 'F#5', '10.11.2.3.5.6', '8.9.0.1.3.4', 'F#4', 'F#4', '6.7.10.11.1.2', '4.5.8.9.11.0', '11.4', '2.3.6.7.9.10', '0.1.4.5.7.8', '10.11.2.3.5.6', '9.1.4', '6.11', 'G#5', 'E-5', '6.11', 'G#4', 'G#4', '6.11', 'F#5', 'F#4', 'F#4', '6.11', '6.11', 'F5', '3.4', 'F4', '3.4', 'F4', '3.4', '6.11', '6.11', '6.11', '6.11', '4.9', 'G#5', 'G#4', 'G#4', '4.9', 'A5', 'A4', 'A4', '4.9', 'G#5', 'G#4', 'G#4', 'E5', 'E4', 'E4', 'B4', 'B3', 'B3', '4.9', 'G#5', 'G#4', 'G#4', '4.9', 'E5', 'E4', 'E4', 'B4', 'B3', 'B3', '4.9', 'G#5', 'G#4', 'G#4', '4.9', 'E5', 'E4', 'E4', '4.9', 'B4', 'B3', 'B3', 'G#4', 'G#3', 'G#3', 'G#4', 'G#3', 'G#3', '11.4', 'A4', 'B-4', 'A3', 'B-3', 'A3', 'B-3', 'A4', 'A3', 'A3', 'B-4', 'B-3', 'B-3', '11.4', 'A4', 'A3', 'A3', '11.4', 'G#4', 'A3', 'A3', '11.4', 'E4', 'E3', 'E3', 'B3', 'B2', 'B2', '11.4', 'A3', 'A2', 'A2', 'G#3', 'G#2', 'G#2', 'G4', 'G3', 'G3', 'A4', 'A3', 'A3', '11.4', '11.4', 'G4', 'G3', 'G3', 'E4', 'E3', 'E3', 'D4', 'D3', 'D3', '11.4', 'B3', 'B2', 'B2', 'A3', 'A2', 'A2', 'G3', 'G2', 'G2', '11.4', 'E3', 'E2', 'E2', 'B4', 'E5', '2.3', '2.3', '2.3', '0.1', '0.1', '0.1', '10.11', '10.11', '10.11', '8.9', '8.9', '8.9', 'F#5', '6.7', '6.7', '6.7', '4.5', '4.5', 'F#5', 'F#5', '11.4', 'E4', '8.11', 'E3', 'G#5', '11.4', '11.4', '11.4', '11.4', '4.8.11', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'B4', '11.4', 'B4', 'B4', '4.8.11', 'C#5', '11.4', '9.1', 'E-5', '11.4', 'E5', '11.4', '4.8.11', 'E5', 'E5', '11.4', 'E-5', '11.4', '11.4', 'E-5', '11.4', 'E-5', 'E-5', 'C#5', '4.8.11', 'C#5', '11.4', 'C#5', 'C#5', '11.4', 'B4', '11.4', 'C#5', '11.4', 'G#4', '11.4', '11.4', '11.4', 'A4', '4.8.11', 'B4', '11.4', 'C#5', '11.4', 'E5', '4.9', 'E4', '9.1', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '6.9', '4.9', 'F#4', '6.9', '4', 'G#5', 'B4', '11.4', '4.8', '8.11', '11.4', '11.4', '9.1', '11.4', '8.11', '11.4', '4.8.11', '11.4', '11.4', '11.4', 'G#5', '11.4', '11.4', '11.4', '11.4', '4.8.11', '4.8', '11.4', '11.4', '3.6', '6.11', '11.3.6', '6.11', '6.11', '6.11', '6.11', '6.11', '6.11', '6.11', '4.9', '9.1.4', '4.9', 'A5', '4.9', 'A5', '4.9', 'G#5', '4.9', 'E5', '4.9', 'E5', '4.9', 'F#5', '4.9', '11.4', '4.8.11', 'E5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '9.11.4', '11.4', '11.4', '11.4', '4.8.11', 'G#5', '4.8.11', 'G#5', '11.2', 'G#5', '11.2.4', 'G#5', '4.8.11', 'G#5', '9.11.4', 'G#5', '6.11', '11.3.6', 'F#5', '6.11', '6.11', 'G#5', '6.11', '6.11', '6.11', '6.11', '6.11', '4.9', '9.1.4', '4.9', '4.9', 'A5', '4.9', 'G#5', '4.9', '4.9', 'E5', '4.9', 'F#5', '4.9', 'E5', '11.4', 'E3', '4.8.11', '11.4', 'B3', 'E5', '11.4', 'E4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'G#5', 'B5', 'G#5', 'F#5', '4.8.11', 'E2', 'B5', 'E5', 'F#5', '4.8.11', 'E2', 'E5', '4.8.11', 'E2', '4.8.11', 'D2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'D2', '9.1.4', 'D2', 'E2', '4.8.11', 'G#2', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'D2', 'G#5', '4.8.11', 'E2', 'B5', '4.8.11', 'B4', '4.8.11', 'E2', 'G#5', '9.1.4', 'E2', 'C#6', '4.8.11', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'G#5', '4.8.11', 'D2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'D2', '4.8.11', 'E2', '4.8.11', 'E2', 'B3', 'F#5', '4.8.11', 'E2', 'E5', '4.8.11', 'E2', '4.8.11', 'D2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'D2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'G#2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'D2', '9.1.4', 'D2', 'E2', '4.8.11', 'G#2', 'B4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'D2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', 'B4', '4.8.11', 'E2', 'B3', 'C#5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B4', '4.8.11', 'E2', 'C#5', '4.8.11', '4.8.11', 'E2', 'B4', '9.1.4', 'E2', 'E5', '4.8.11', 'E2', '4.8.11', 'E2', 'B4', '4.8.11', 'E2', '4.8.11', 'E2', 'G#5', '4.8.11', 'E2', '4.8.11', 'E2', 'E5', '4.8.11', 'F#5', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', 'E5', '4.8.11', 'E2', '4.8.11', 'G#2', '4.8.11', 'E2', '4.8.11', 'B1', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B5', '4.8.11', 'E2', '4.8.11', '4.8.11', 'D2', '9.1.4', 'D2', 'E2', '4.8.11', 'G#2', 'E2', '4.8.11', 'E2', 'B5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'B1', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'D2', '9.1.4', 'D2', 'E2', 'B5', '4.8.11', 'G#2', 'E2', '4.8.11', 'E2', 'B5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'B1', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B4', '4.8.11', 'G#1', '4.8.11', '4.8.11', 'B1', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B3', 'G#5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', 'G#5', '9.1.4', 'E2', 'F#5', '4.8.11', 'E2', 'E5', 'E5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'E5', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', 'E-5', '9.1.4', 'E2', 'C#5', '4.8.11', 'E2', 'B4', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', '4.8.11', '4.8.11', 'D2', '9.1.4', 'D2', 'E2', '4.8.11', 'G#2', 'E2', '4.8.11', 'E2', '4.8.11', 'E2', 'B4', '4.8.11', 'E2', 'C#5', '4.8.11', 'E2', 'E5', 'F#5', '4.8.11', 'E2', '4.8.11', '4.8.11', 'E2', '9.1.4', 'E2', '4.8.11', 'E2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'G2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', '4.7', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', '4.7', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A3', 'A2', '2.6', 'A2', '4.7', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A3', 'A2', '2.6', 'A2', '4.7', 'A2', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A1', 'A2', 'A2', 'A1', 'A2', 'A2', 'A1', 'A2', 'A2', 'A2', 'A1', 'A2', 'A1', 'A2', 'A2', 'A2', 'A2', 'A1', '4.7', 'A2', 'A2', '2.6', 'A2', 'A2', 'A1', 'A2', 'A2', 'A2', 'A1', '3.7', 'A2', 'A2', '0.4', 'A2', 'A1', 'A3', 'A2', 'A1', '2.6', 'A2', '0.4', 'A2', 'A2', 'A1', 'G3', 'A2', 'E3', 'A2', 'A2', 'A1', 'G3', 'A2', 'A3', 'A2', 'A2', 'A1', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A1', '1.4', 'C#4', 'A2', 'A3', 'A2', 'E3', 'A2', '1.4', 'A2', 'A1', '4.7', 'A2', 'A2', '2.6', 'A2', 'A2', 'A1', 'A2', 'A2', 'A2', 'A1', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', 'A1', '2.6', 'A2', '0.4', 'A2', 'A2', 'A1', 'G3', 'A2', 'A2', 'A1', 'E3', 'A2', 'C2', '7.0', '7.0', '7.0', '7.0', 'D2', '9.2', '9.2', '9.2', '9.2', 'G1', '2.7', '2.7', '2.7', '2.7', 'C2', '7.0', '7.0', '7.0', '7.0', 'F2', 'F2', '0.5', '0.5', '0.5', '0.5', 'F2', 'F2', 'F2', '0.5', '0.5', 'F2', '0.5', 'F2', 'F2', 'F3', 'F2', 'C4', 'C3', 'A2', 'D4', 'F2', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'E-4', 'F2', 'F2', 'E-4', 'F2', 'F2', 'F2', 'F2', 'D4', 'F2', 'F2', 'C4', 'F2', 'F2', 'C4', 'F2', 'B-2', 'B-1', 'D4', 'B-2', 'B-2', 'C4', 'B-2', 'B-2', 'B-1', 'B-3', 'B-2', 'D4', 'B-2', 'F2', 'F2', 'F2', 'F2', 'C4', 'F2', 'F2', 'F2', 'C4', 'F2', 'D4', 'F2', 'F2', 'F2', 'E-4', 'F2', 'E-4', 'F2', 'F2', 'F2', 'E-4', 'E-4', 'F2', 'F2', 'A2', 'E-4', 'F2', 'F2', 'D4', 'F2', 'A2', 'C4', 'F2', 'C4', 'F2', 'F2', 'B-2', 'B-2', 'D4', 'B-2', 'F4', 'B-2', 'B-2', 'D4', 'B-2', 'D2', 'B-3', 'B-2', 'E-2', 'B-2', 'C4', 'F2', 'F2', 'F2', 'F2', 'A3', 'F2', 'F2', 'C4', 'F2', 'D4', 'F2', 'C2', '7.0', 'E4', '7.0', '7.0', '7.0', 'E4', 'C2', 'E4', 'E4', 'E4', 'C2', 'D4', 'E4', 'C4', 'C4', 'C2', 'D4', 'G3', 'G3', 'E4', 'F4', 'C#2', '8.1', '8.1', '8.1', '8.1', 'F4', 'G#2', 'E-4', 'C#4', 'F4', 'C#3', 'G#2', '8.1', '8.1', '8.1', '8.1', 'F4', 'F4', 'G#1', '3.8', '3.8', '3.8', '3.8', 'E-4', 'E-4', 'E-2', 'E-4', 'F4', 'G#2', 'E-4', 'C4', 'E-2', 'E-4', 'E-4', 'E4', 'E4', 'C4', 'G3', 'C4', 'G3', 'E2', '0.4.7', 'C3', 'C3', '7.0', 'E4', 'E2', 'E4', 'E2', 'E4', 'F2', 'F#2', 'E-4', 'E4', 'G2', 'C2', 'C4', '7.0', '7.0', '7.0', 'C4', 'F2', 'F2', '5', 'E-4', '0.5', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'E-4', 'F2', 'E-4', 'F2', 'F2', 'F2', 'E-4', 'F2', 'D4', 'F2', 'C4', 'F2', 'F2', 'C4', 'F2', 'F2', 'B-2', 'B-1', 'D4', 'B-2', 'B-2', 'F4', 'B-2', 'B-2', 'B-1', 'D4', 'B-3', 'B-2', 'B-2', 'C4', 'F2', 'F2', 'F2', 'F2', 'F4', 'F2', 'F2', 'F2', 'F4', 'F2', 'F2', 'F2', 'F2', 'A3', 'F2', 'C4', 'F2', 'C4', 'F2', 'F2', 'F2', 'F2', 'D4', 'F2', 'F2', 'A2', 'C4', 'F2', 'C4', 'F2', 'F2', 'A2', 'D4', 'F2', 'F4', 'F2', 'F2', 'D4', 'B-2', 'B-2', 'B-2', 'F4', 'B-2', 'D2', 'B-2', 'D2', 'D4', 'B-3', 'B-2', 'B-2', 'C4', 'F2', 'F2', 'F2', 'F2', 'F2', 'F2', 'C4', 'F2', 'D4', 'F2', 'C2', '7.0', 'E4', '7.0', '7.0', '7.0', 'E4', 'C2', 'E4', 'E4', 'C2', 'D4', 'E4', 'E4', 'C2', '7.0', '7.0', '7.0', 'C#2', '1.5.8', 'F4', '8.1', '8.1', '8.1', 'F4', 'G#2', 'E-4', 'C#4', 'C#3', 'F4', '8.1', '8.1', 'G#2', '8.1', '8.1', 'E4', 'F4', '8.1', '8.1', 'G#2', 'G#1', '3.8', '3.8', 'E-4', 'E-4', 'G#2', 'G#1', 'E-4', 'F4', 'G#2', 'G#2', 'G#1', 'E-4', 'C4', 'G#2', 'G#3', 'G#2', 'G#1', 'G#2', 'A2', 'A1', '4.9', 'A3', 'E4', 'E4', 'A2', 'A1', 'C#4', 'E4', 'E4', 'E4', 'A2', 'A1', '4.9', 'A4', 'E4', 'A2', 'A1', 'D2', '2.6.9', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'D2', '9.2', '9.2', 'F#4', 'G1', '2.7', 'E4', '2.7', '2.7', '2.7', 'D4', 'D2', '2.6.9', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'A1', '4.9', '4.9', '4.9', '4.9', 'F#4', 'A1', '4.9', '4.9', 'D2', 'A3', 'F#4', '9.2', '9.2', 'A3', 'F#4', 'C#2', 'B-3', 'E4', '1.4.6', '1.4.6', 'B-3', 'E4', 'B1', 'D4', 'E4', '6.11', '6.11', 'D4', 'D4', 'D4', 'G1', 'D4', 'E4', '2.7', '2.7', 'D4', 'F#4', 'A1', 'B3', 'A3', '4.9', '4.9', '4.9', 'A3', 'D4', 'A1', 'E4', 'B3', '4.9', '4.9', '4.9', 'D4', 'D4', 'D2', 'D4', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', '4.7', 'A2', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', 'A2', '4.7', 'A2', 'A2', '2.6', 'A2', 'A2', 'A2', 'A2', 'A2', '3.7', 'A2', 'A2', '0.4', 'A2', 'A3', 'A2', '2.6', 'A2', '0.4', 'A2', 'A2', 'G3', 'A2', 'A2', 'E3', 'A2', 'A2', 'G3', 'A2', 'A3', 'A2', 'A2', 'A1', 'A2', 'A2', 'A1', 'A2', 'A2', 'A1', 'A2', 'A2', 'A2', 'A1', 'A2', 'A1', 'A2', 'A2', 'A2', '1.4.7', 'A2', 'A1', '4.9', 'A2', 'A2', '0.4.7', 'A2', 'A1', '1.4.7', '4.9', 'A2', '11.2.6', 'A2', 'A2', 'A1', '1.4', '4.9', 'A2', 'A3', 'A2', 'A2', 'A1', '9.1', '4.9', 'A2', 'A2', 'A2', 'A1', '1.4.7', '4.9', 'A2', 'A2', '0.4.7', 'A2', 'A1', '1.4.7', '4.9', 'A2', '11.2.6', 'A2', 'A2', 'A1', '1.4', '4.9', 'A2', 'A3', 'A2', 'A2', 'A1', '9.1', '4.9', 'A2', 'A2', 'A2', 'A1', '1.4.7', '4.9', 'A2', 'A2', '1.4.7', 'A2', 'A1', '1.4.7', '4.9', 'A2', 'A2', '1.4.7', 'A2', 'A1', '6.9.1', '4.9', 'A2', '6.9.1', 'A2', '6.9.1', 'A2', 'A1', '6.9.1', '4.9', 'A2', 'A2', 'A2', 'A1', '1.4.7', '4.9', 'A2', 'A2', '4.7.11', 'A2', 'A1', '4.7', 'B4', '4.9', '4.7', 'A2', '4.7.11', 'A2', '4.7.11', 'A2', 'A1', '9.1.4', '4.9', 'A2', 'A2', 'A2', 'A2', 'A1', '4.9', 'A2', 'A2', 'A2', 'A2', 'A1', '4.7', '4.9', 'A2', '2.6', 'A2', 'A2', 'A1', '4.9', 'A2', 'A2', 'A2', 'A1', '3.7', '4.9', 'A2', '0.4', 'A2', 'A3', 'A2', 'A1', '2.6', '4.9', 'A2', '0.4', 'A2', 'A2', 'A1', 'G3', '4.9', 'A2', 'A2', 'E3', 'A2', 'C2', '7.0', '7.0', '7.0', '7.0', 'D2', '9.2', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', '9.2', 'E-2', 'E2', '9.2', '9.2', '9.2', 'F2', 'F2', '0.5', '0.5', '5.9.0', '0.5', '5.9.0', '0.5', 'F2', 'F2', 'F2', 'F2', '0.5', '5.9.0', '0.5', '5.9.0', 'F2', 'F2', 'F2', 'C4', 'C4', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'E-4', 'F2', 'E-4', 'F2', 'F2', 'F2', 'E-4', 'D4', 'F2', 'F2', 'C4', 'F2', 'F2', 'F2', 'F2', 'C4', 'F2', 'B-2', 'B-1', 'D4', 'B-2', 'C4', 'B-2', 'B-2', 'B-3', 'B-2', 'B-1', 'D4', 'B-2', 'B-2', 'C4', 'F2', 'F2', 'F2', 'F2', 'F2', 'A3', 'F2', 'F2', 'C4', 'F2', 'D4', 'F2', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'F2', 'F2', 'E-4', 'E-4', 'F2', 'E-4', 'F2', 'A2', 'E-4', 'F2', 'D4', 'F2', 'C4', 'F2', 'A2', 'F2', 'F2', 'F2', 'C4', 'B-2', 'B-2', 'D4', 'B-2', 'F4', 'B-2', 'B-2', 'B-2', 'D2', 'F4', 'B-2', 'E-2', 'B-2', 'F4', 'F2', 'F2', 'D4', '10.2.5', '10.2.5', 'C4', 'F2', 'C4', '0.5', '0.5', 'D4', 'C2', '7.0', 'D4', '7.0', '7.0', '7.0', 'E4', 'E4', 'C2', 'E4', 'E4', 'C2', 'C4', 'C2', 'E4', '7.0', '7.0', 'E4', 'C#2', '8.1', 'F4', '1.5.8', '1.5.8', '8.1', 'F4', 'E-4', 'G#2', 'C#4', 'F4', 'C#3', 'E-4', 'G#2', '8.1', 'C#4', '1.5.8', '1.5.8', '8.1', 'F4', 'F4', 'G#1', 'G#2', '3.8', '8.0.3', '8.0.3', '3.8', 'E-4', 'E-4', 'E-2', 'E-4', 'F4', 'G#2', 'E-4', 'C4', 'E-2', '8.0.3', '8.0.3', 'E-4', 'E-4', '3.8', '3.8', 'C2', '0.4.7', '0.4.7', '0.4.7', '7.0', 'E4', 'C2', 'E4', 'C2', 'E4', 'C2', 'F4', 'C2', '0.4.7', 'G4', 'C2', 'E4', '0.4.7', '0.4.7', '7.0', 'C4', 'E4', 'F2', 'F2', '5', 'F4', '0.5', 'F2', 'F4', 'F2', 'F2', 'D4', 'C4', 'F2', 'F2', 'F4', 'F2', 'F2', 'F2', 'F2', 'F4', 'F2', 'F2', 'D4', 'F2', 'C4', 'F2', 'D4', 'B-2', 'B-1', 'B-2', 'B-2', 'F4', 'B-2', 'B-2', 'B-1', 'F4', 'F4', 'B-2', 'B-2', 'F4', 'F2', 'F2', 'F2', 'F2', 'F4', 'F2', 'F2', 'F2', 'F2', 'F4', 'F2', 'F2', 'F2', 'A3', 'F2', 'C4', 'F2', 'F2', 'F2', 'C4', 'F2', 'F2', 'D4', 'F2', 'F2', 'F2', 'C4', 'F2', 'C4', 'F2', 'F2', 'F2', 'D4', 'F2', 'F2', 'F4', 'F2', 'B-2', 'B-1', 'B-4', 'B-2', 'A4', 'B-2', 'F4', 'B-2', 'B-1', 'B-2', 'B-2', 'F4', 'B-2', 'B-2', 'F2', 'F2', 'F3', 'D4', 'F2', 'C3', 'F2', 'F3', 'C4', 'F2', 'F2', 'F2', 'C4', 'F2', 'F2', 'F2', 'D4', 'F2', 'F2', '7.0', 'C2', '0.4', 'E4', '7.0', '7.0', '7.0', 'E4', 'G2', 'E4', 'E4', 'E4', 'C3', 'D4', 'E4', 'G2', 'D4', '7.0', '7.0', '7.0', 'E4', 'F4', 'C#2', '1.5.8', '8.1', '8.1', '8.1', 'F4', 'G#2', 'E-4', 'C#4', 'F4', 'C#3', '8.1', '8.1', 'F#4', 'G#2', '1.5.8', '8.1', '8.1', 'F4', '8.1', '8.1', 'G#2', 'G#1', '3.8', '3.8', 'E-4', 'E-4', 'G#2', 'E-2', 'E-4', 'F4', 'G#2', 'G#2', 'G#2', 'E-4', 'C4', 'G#2', 'G#2', 'E-2', '3.8', 'G#2', 'A2', 'A1', '4.9', 'A3', 'E4', 'E4', 'G3', 'A2', 'A1', 'E3', 'E4', 'C#3', 'F#4', 'B2', 'A2', 'A1', '4.9', 'A2', 'E4', 'C#4', 'A2', 'A1', 'D2', '2.6.9', '9.2', '9.2', 'D3', 'D2', '9.2', '9.2', 'D2', '9.2', '9.2', 'F#4', 'G1', '2.7', 'E4', '2.7', '2.7', 'D4', 'D2', '2.6.9', '9.2', '9.2', 'D2', '9.2', '9.2', 'A1', '4.9', '4.9', '4.9', 'F#4', 'A1', '4.9', '4.9', 'C#4', 'D2', 'A3', 'F#4', '9.2', '9.2', 'A3', 'F#4', 'F#4', 'F#4', 'C#2', 'B-3', 'E4', '1.4.6', '1.4.6', 'B-3', 'E4', 'E4', 'E4', 'B1', 'D4', 'E4', '6.11', '6.11', 'D4', 'E4', 'D4', 'D4', 'D4', 'D4', 'G1', 'D4', 'E4', '2.7', '2.7', 'D4', 'E4', 'F#4', 'F#4', 'A1', 'E4', 'A3', '4.9', '4.9', 'A3', 'A3', 'A3', 'A3', 'D4', 'A3', 'A1', 'A3', 'B3', '4.9', '4.9', 'A3', 'B3', 'A3', 'D4', 'A3', 'D4', 'D4', 'D4', 'D2', 'D4', '9.2', '9.2', 'D4', 'D2', '9.2', '9.2', 'A3', 'C4', '0.5', 'F2', 'F2', 'C3', '0.5', 'F3', '0.5', 'F2', '0.5', 'F2', '5.10', 'B-1', 'C4', 'F2', 'C4', '0.5', 'F2', 'C5', 'F3', 'F4', '0.5', 'D5', 'F4', 'E-5', 'F3', 'F#4', '7.0', 'C2', 'E5', 'C3', 'G4', '7.0', 'E5', 'G4', '0.5', 'F2', 'C5', 'C3', 'C4', 'D5', '4.9', 'E2', 'G4', 'A2', 'C#4', 'G4', '9.2', 'D2', 'F4', 'D3', 'D4', 'D5', '5.10', 'B-1', 'B-2', 'D4', 'B-2', 'D4', 'C5', 'B-4', '7.0', 'C2', 'G4', 'C3', 'E4', 'A4', 'C3', '7.0', 'C2', 'A4', 'B-2', 'E4', 'D4', 'B-2', 'D4', '0.5', 'F1', 'F4', 'A2', 'C4', '0.5', 'F1', 'G4', '0.5', 'F1', 'A4', 'F3', 'C4', 'A4', 'F3', 'C4', '0.5', 'F1', 'G4', 'F3', 'C4', 'D4', 'F3', 'C4', '0.5', 'F1', 'F4', 'F3', 'C4', '5.10', 'B-1', 'C3', 'B-3', '0.5', 'F1', 'A4', 'C3', 'C4', 'A4', 'C3', 'C4', '0.5', 'F1', 'G4', 'C3', 'C4', 'D4', 'C3', 'C4', '7.0', 'C2', 'A4', 'C3', 'C4', 'G4', '7.0', 'C2', '0.5', 'F2', 'A4', 'C3', 'C4', 'C5', '4.9', 'E2', 'A4', 'A2', 'C#4', 'C5', '9.2', 'D2', 'A4', 'D3', 'D4', 'C5', 'D4', '5.10', 'B-1', 'A4', 'B-2', 'F4', 'B-2', 'C5', 'D4', '7.0', 'C2', 'G4', 'C3', 'C4', 'D4', 'A2', 'A3', '7.0', 'C2', 'C4', 'C3', 'C4', 'D4', 'C3', 'C4', '0.5', 'F2', 'F4', 'F2', '9.0', '0.5', 'A1', '4.9', '4.9', '4.9', 'D2', '2.6.9', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'D2', '9.2', '9.2', 'A4', 'G1', '2.7', '2.7', '2.7', '2.7', 'D4', 'D2', '2.6.9', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'A1', '4.9', '4.9', '4.9', '4.9', 'F#4', 'A1', '4.9', '4.9', 'D2', 'A3', 'F#4', '9.2', '9.2', 'A2', 'F#4', 'C#2', 'B-3', 'E4', '1.4.6', '1.4.6', 'F#2', 'E4', 'B1', 'D4', 'E4', '6.11', '6.11', 'B2', 'D4', 'D4', 'G1', 'D4', 'E4', '2.7', '2.7', 'G2', 'F#4', 'E4', 'A1', 'E4', 'E4', '4.9', '4.9', '4.9', 'D4', 'E4', 'A1', 'A3', 'E4', '4.9', '4.9', '4.9', 'F#4', 'A3', 'D4', 'D2', 'D4', '9.2', '9.2', '9.2', 'D2', '9.2', '9.2', 'A2', '4', 'E2', '11.4', '4', '4', '4', 'A2', 'A2', 'A2', 'A2', 'F#4', '10.2', 'A4', '9.1', 'A4', 'G4', 'E4', 'C#4', 'E4', 'C#4', 'E4', 'E4', 'A3', 'G#3', 'B3', 'E4', 'A3', 'C#4', 'F#4', 'B3', 'D4', 'G#4', 'C#4', 'E4', 'A4', 'D4', 'F#4', 'B4', 'E4', 'G#4', 'C#5', 'F#4', 'A4', 'D5', 'G4', 'B-4', 'E-5', 'G#4', 'B4', 'E5', 'A4', 'C#5', 'F#5', 'B4', 'D5', 'G5', 'C5', 'E-5', 'G#5', 'C#5', 'E5', 'A5', '2.6.9', '9.2', '9.2', '9.2', '9.2', '9.2', '9.2', '9.2', 'F#4', '2.7', 'E4', '2.7', '2.7', '2.7', 'D4', '2.6.9', '9.2', '9.2', '9.2', '9.2', '9.2', '4.9', '4.9', '4.9', '4.9', 'F#4', '4.9', '4.9', 'F#4', 'A3', 'F#4', '9.2', '9.2', 'A2', 'F#4', 'B-3', 'F#4', '1.4.6', '1.4.6', 'F#2', 'F#4', 'D4', 'F#4', '6.11', '6.11', 'B2', 'F#4', 'D4', 'F#4', '2.7', '2.7', 'G2', 'A4', 'B3', 'F#4', '4.9', '4.9', '4.9', 'D4', 'F#4', 'E4', 'E4', '4.9', '4.9', '4.9', 'D4', 'D4', 'D4', '9.2', '9.2', '9.2', '9.2', '9.2', '9.2', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', 'A4', '5.10', 'G4', '5.10', '5.10', '5.10', 'F4', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.4.7', '7.0', '7.0', '7.0', '7.0', '7.0', 'C4', 'A4', '0.5', '0.5', 'F2', 'C5', 'C4', 'G4', '4.9', '4.9', 'C3', 'C5', 'C4', 'F4', '9.2', '9.2', 'D3', 'C5', 'C4', 'D4', '5.10', '5.10', 'B-2', 'A4', 'C4', 'C4', '7.0', '7.0', '7.0', 'A3', 'C4', 'F3', 'D4', '7.0', '7.0', '7.0', 'F3', 'F4', 'F4', 'F3', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '2.6.9', '9.2', '9.2', '9.2', '9.2', '9.2', '9.2', '9.2', 'F#4', 'A4', '2.7', 'F#4', '2.7', '2.7', '2.7', 'A4', '2.6.9', 'F#4', '9.2', '9.2', '9.2', 'A4', 'F#4', 'A4', '9.2', '9.2', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', 'A3', 'F#4', '9.2', '9.2', 'A2', 'F#4', 'F#4', 'F#4', 'B-3', 'E4', '1.4.6', '1.4.6', 'F#2', 'E4', 'E4', 'E4', 'D4', 'E4', '6.11', '6.11', 'B2', 'E4', 'D4', 'D4', 'D4', 'D4', 'D4', 'E4', '2.7', '2.7', 'G2', 'E4', 'F#4', 'F#4', 'F#4', 'A3', '4.9', '4.9', '4.9', 'A3', 'A3', 'A3', 'E4', 'F#4', 'B3', '4.9', '4.9', '4.9', 'B3', 'D4', 'D4', 'E4', 'D4', 'D4', 'F#4', '9.2', '9.2', '9.2', 'F#4', '9.2', '9.2', '9.2', 'E4', 'B2', 'D4', '6.11', 'B2', '6.11', 'B2', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', 'B-3', '6.11', 'B2', 'B3', '6.11', 'B2', 'B3', '6.11', 'B3', 'B2', 'E3', 'F#4', '11.4', 'E3', '11.4', 'E3', '11.4', 'B2', 'B4', '6.11', 'B2', 'A4', '6.11', 'B2', 'B2', 'B4', '6.11', 'B2', 'B-4', '6.11', 'B2', '6.11', 'F#2', '1.6', 'F#2', 'E-4', '1.6', 'E-4', 'F#2', 'F#2', 'F#4', '1.6', 'F#2', '1.6', 'F#2', '1.6', 'B2', 'F#4', 'F#4', '11.3', '6.11', 'B-2', 'F#4', 'E4', '10.3', '6.11', 'G#2', 'F#4', 'E-4', '4.9', '6.11', 'E2', 'F#4', 'F#4', '4.8', '6.11', 'F#2', 'E4', 'E-4', '1.6', 'F#2', 'D4', 'D4', '1.6', 'F#2', 'F#2', 'E4', 'E-4', '1.6', 'F#2', 'D4', 'C#4', '1.6', 'F#2', 'B3', '1.6', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', '6.11', 'E-4', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', 'E-4', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', 'E-4', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'E-4', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B3', '6.11', 'B2', '6.11', 'B2', 'E-4', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', 'E-4', '6.11', 'B2', 'F#4', '6.11', 'B2', '6.11', 'B2', 'E-4', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', 'E4', '6.11', 'B2', '6.11', 'E-4', '6.11', 'B2', '6.11', 'B2', 'D4', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', '6.11', 'B2', 'G1', 'G1', 'G1', 'G1', 'G1', 'G3', 'B-3', 'G1', 'B-3', 'C4', '0.3.7', 'C4', '0.3.7', 'B-3', 'C4', '0.3.7', 'G3', '0.3.7', '2.5.8.10', 'D4', 'C4', 'D4', '2.5.8.10', 'E-4', 'D4', '2.5.8.10', 'C4', 'C4', '5.10', 'G#4', 'D4', 'G2', 'C#6', 'F2', 'C#6', '0.3.7', 'C3', 'C4', 'C#6', 'C2', 'C4', 'B-3', '0.3.7', 'C4', '0.3.7', 'G3', '0.3.7', 'G3', 'D4', '2.5.8.10', 'D4', '8.10.2', 'F4', 'D4', 'D4', 'E-4', 'C3', '3.7.10', 'B-2', 'F4', 'G4', 'B-2', 'C2', 'D2', 'D2', '3.7.10', '0.1', '7', '5.10', 'C#6', '3.7.10', 'E-2', '0.1.6', 'G4', 'E-2', 'E-4', 'G4', 'E-4', '3.7.10', 'E-2', '2.6', 'E-4', 'E-3', 'F4', 'D2', '0.1.6', '7.10.2', 'D2', 'D4', 'E-4', 'F4', 'D4', 'D4', '2.6', '7.10', '2', 'D3', 'C#4', '0.1.6', '7.10.1', '1.3', 'C#2', 'E-4', 'C#4', 'G4', '10.1', '2.6', 'C#2', 'E-4', '1', '8.0.3', '0.1.6', '8.0.3', 'C4', 'C4', 'E-4', 'G#1', 'G#3', 'G#1', 'E-4', 'C4', '2.6', 'C4', '3.8', '0.3', 'G#1', 'G#2', 'C#6', 'C4', '11', '0.1.6', '8.11.3', 'G#1', 'E-4', 'G#4', 'B4', 'G#1', 'E-4', 'B3', 'B3', 'E-4', '2.6', 'G#1', '8.11', 'B3', 'C#6', 'G#2', 'B-3', 'B-3', '7.10', 'B-1', '0.1.6', '7.10', 'B-1', 'E-4', 'B-3', 'B-3', 'B-3', '2.6', '10.3', '7.10', 'B-3', 'F2', 'F1', '0.1.6', '10', '8.0.3', 'C5', 'G#4', 'E-4', 'B-2', 'B-3', 'B-3', 'C2', 'C2', 'B-3', '1.2.6', '3.8', 'C4', 'B-1', 'B-3', 'F1', 'F2', 'G4', '0.1', '10.2.5', 'D4', 'B-1', 'G#3', '10', 'G#3', 'E-2', '2.5', 'F#2', 'F4', 'G4', 'G1', 'G2', 'G2', '0.1.6', 'G4', 'F4', 'G3', 'F#2', 'G4', '2.6', '11.2.5', 'G4', 'F#2', '7', 'E-4', 'C2', '0', '0.6', 'D4', 'C4', '0.6', 'G3', '2.6', '0.3', '7.0', 'F#2', '10', '0.1.6', 'B-2', 'B-4', 'G#4', 'B-4', 'F3', 'F#2', 'B-3', '2.6', 'D4', 'G#4', 'F3', 'B-2', 'F#2', 'G4', '0.1.6', 'E-2', 'E-3', 'E-2', 'F4', 'E-4', '0.6', 'G3', '10.3', '2.6', 'G3', 'F#2', 'E-2', 'D3', '2.6', 'D2', '6.9.0', '0.1.6', 'D2', 'B-3', 'A3', '6.9.0.2', '1.2.6', 'G3', 'F#3', '2', 'D2', '0.1.6', '7.10', '7.10.2', 'G2', '9.0', '2.8', '5.9.0', 'F2', 'F2', '5', '5.9.0', 'D4', '0.6', '10.2.5', '10', '10.2.5', '10.2', 'B-3', 'F4', 'F4', '2.6', 'D4', '5.10', 'C#6', '10', '0.1.6', '2', '2.5.9', 'F4', '2', 'A4', 'D2', '5', 'F4', '9.2', '2.6', 'F5', 'C5', '2.5', 'B-4', 'G2', '0.6', '7.10.2', '7.10.2', 'G2', 'B-4', 'C#6', '2.7', 'B-4', '1.2.6', '7.10.2', 'A4', '2.5.9', 'C#6', 'F2', '9.2', '2.5.9', '0.1.6', 'D2', 'D2', '5', '2.5', 'D4', 'F4', 'F4', 'C2', 'A3', '2.6', '2.5.9', 'F2', 'C#6', 'F2', 'B-4', '0.1.6', 'G2', '7.10.2', 'G2', '10.2', 'G4', '7.10.2', '7.10.2', '2.6', '2.5.9', 'A4', 'A4', 'F2', '2.5.9', 'F2', 'C#6', '4.7', '0.1.6', '9.1.4', 'A1', '1.7', '4.9', 'A3', 'F4', '5.9', '2.5', 'D4', '1.2.6', '2', 'G4', '0', 'C#6', '0.1.6', '7.11', '5.7', '5.7', 'G2', 'B3', 'G1', 'A4', 'G4', '5.9.0', '1.2.6', '5.9.0', 'A2', 'A2', 'G4', 'E4', '0.1.6', '11.2', '5.11', '11.2.5', 'E4', 'D4', '0.4.7', '0.4.7', '1.2.6', 'G4', 'E4', 'C3', 'E3', 'D2', 'D2', 'D2', '1.2', 'G3', '0.1', '10.2', 'F3', 'F3', '10', '10.2.5', '10.2.5', 'F4', 'D5', 'B-4', 'F3', 'B-4', 'B-3', 'D4', 'F4', '0.1.6', 'A1', 'A2', 'A1', 'A4', 'A4', 'A3', 'G4', 'A4', '9.1', '2.6', 'E4', '1.4', 'A4', 'A3', 'C#6', '9', 'F4', '0.1.6', 'D2', 'D3', 'D2', 'E4', 'D4', 'C2', 'A3', '2.5', '2.5.9', '2.6', 'G3', 'A3', 'D2', 'G2', '0.6', 'G4', 'G1', 'G1', 'G4', 'G3', '2.6', '11.2.5.7', '2.7', 'B3', 'F4', '7', 'C#6', 'C3', '0.1.6', 'C2', 'E4', 'C2', 'D4', '7.0', 'C2', '0.4', '0.2.4.6', '4.7', 'E4', 'G3', '0.6', '0.4', '0', 'C2', 'B-1', 'F3', '0.1', 'B-1', 'B-2', '10.2', '10.2', '2.5', 'C4', '0.3.7', '0.3', 'D4', 'D5', 'D4', 'F4', 'B-4', 'F3', 'B-3', 'D4', 'C#6', 'F4', 'C#6', '0.5', '5', 'F2', 'F1', 'C3', 'F#2', 'F2', 'A4', 'F3', '0.5', 'A3', 'F#2', 'F3', 'F#2', 'D4', 'B-3', 'F3', '7.10', 'F#2', '3.7.10', '3.5.7', 'G4', 'G3', '7.10', '10.3', 'G3', 'B-3', 'F2', 'E-4', 'G4', 'F#2', 'F3', 'F4', 'F#2', 'F3', 'D5', '2.5', 'F#2', '10.2', '7.10.2', 'G3', 'B-4', 'A2', '10.3', 'E-4', 'G5', 'E-5', '6.9', 'G2', 'G2', '0.1', '0.5', '5', 'F1', 'F3', '9.0', '2.6', 'F4', 'F2', 'F1', 'C3', 'F3', '0.6', 'B-1', '10', '5.10', 'D4', 'C2', 'F3', '3.7', 'B-3', '3.7.10', '2.6', 'G3', '10.3', 'B-3', 'G3', 'E-4', 'G3', '7.10', 'B-3', 'E-4', 'G4', 'B-3', '5', '0.6', 'C#3', 'F3', 'C#3', 'C#3', 'F4', 'C#3', 'C#3', 'C#3', 'C#3', 'C#3', 'C#3', 'F3', 'C#3', 'C#3', 'C#3', '10', 'D4', 'B-2', 'C#3', 'C#3', 'C#3', 'C#3', 'C#3', 'C#3', 'F2', 'C#3', 'C#3', '7', 'C#3', '3.7.10', 'C3', 'G1', 'C#3', 'C#3', 'C#3', 'B-2', 'C#3', 'C#3', 'C#3', 'C3', 'G3', 'C#3', 'C#3', 'C#3', 'C4', '0.3.7', 'C3', 'C2', 'C4', '0.3.7', 'F#2', 'B-3', 'C4', '0.3.7', 'F#2', 'G3', 'F#2', '3.7', 'C4', 'G3', '8.10', '10.2.5', 'F#2', 'D4', 'D2', 'C4', 'D2', 'D2', '2.6', '5.8.10', '10.2.5', 'D4', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'E-4', 'D2', 'D2', 'D4', '2.6', '2.5.8.10', 'D2', 'D2', 'D2', 'D2', 'D2', 'C4', 'D2', 'D2', 'D2', 'C4', '2.6', 'G#4', '2.5', 'B-4', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'C3', '10.0.2.6', '0.3.7', 'C3', 'C2', 'C3', 'A5', 'B-3', 'C4', '0.3.7', 'F#2', 'B-3', 'C4', '0.3.7', 'F#2', 'G3', '0.3.7', 'F#2', 'G3', 'D4', '2.5.8.10', 'F#2', 'D4', 'F#2', '2.5.8.10', 'D4', 'D4', 'E-4', 'F4', '3.7.10', 'C3', 'F#2', 'F4', 'C2', 'G2', 'G4', 'F4', 'C2', 'B-2', '2.6', '3.7', '0.5', '10.3', 'E-4', '7', 'C2', 'C#6', 'F2', 'F2', '3.7.10', '0.1.6', 'B-3', 'E-2', '3.7', '3.7', 'E-2', 'B-4', 'G4', 'G4', 'E-2', '3.7.10', '2.6', 'E-4', 'E-4', 'E-3', 'F4', 'D2', '7.10.2', '0.1.6', '2.5', '10', 'D2', 'F4', 'D4', 'B-4', 'G4', '10.2', '2.6', 'D4', 'D3', '7.10.1', '1.3', 'C#2', '0.1.6', 'G3', 'G4', 'E-4', 'B-3', 'C#4', 'E-4', 'E-4', '2.6', '10.1', '7.10', 'C#4', 'C#2', '1.3', '7.10', '3', '8.0', '0.1.6', '8.0.3', 'G#1', '0.3', 'C4', 'E-4', '8.0.3', 'G#4', '8.0', 'C4', '2.6', 'G#1', 'E-4', 'C#6', '8.11.3', '8.11.3', '0.1.6', 'G#1', 'B3', '8.11.3', 'G#1', '3', 'B3', 'E-4', 'B3', 'E-4', 'G#4', '8.11', '1.2.6', 'E-4', '8.11', 'B3', 'B3', 'G#1', 'B3', 'G#2', '10', '7.10', '0.1.6', 'B-1', 'E-4', 'B-3', 'B-3', 'G3', '10', '3', '7.10', '1.2.6', '10.3', 'B-3', '3.5', '5.7', 'B-3', 'B-3', '8.0.3', '8.10.0.3', 'B-2', 'B-1', '0.1.6', 'B-3', 'C2', 'C2', '8.0', '0.3', 'G#3', 'B-3', '10', '1.2.6', '0.3', 'G4', '5', '5.8.10', 'D4', '0.1', '10.2.5', '2.5.8.10', 'B-1', 'G4', 'G4', 'G4', 'G4', '2.5', '7.11.2', '0.1.6', 'B4', 'G1', 'G4', 'G3', 'G1', 'G4', 'G4', '11.2.5', '2.6', '7.11', 'G4', '7', 'C#6', 'C2', '0.3.7', '0.1.6', 'C2', 'E-4', 'C4', 'C2', '2.6', '0.3.7', '0.3.7', '10.2.5', 'G3', '0.3.7', 'C2', 'C#6', 'B-3', '5.10', '0.1.6', '10.2', 'B-2', 'B-1', 'B-3', 'G2', '2.6', '10.2.5', 'G#2', 'B-3', 'G#3', 'B-1', 'C#6', 'F2', 'G3', '0.1.6', '3.7.10', 'E-2', 'E-2', 'E-4', 'C2', '2.6', '10.3', '7.10', '3.7', 'C#6', 'A4', '0.1.6', '6.9.0', 'F#3', 'D4', 'D2', 'G3', 'F3', 'G3', 'E-3', '6.7', '2.6', '2.6.9', '9.0.2', 'C3', 'G3', 'B-2', 'D2', 'C#6', 'G3', '7.10.2', 'G2', '0.1.6', 'B-4', 'C2', 'C4', '5.9.0', 'F2', '5.9.0', '2.6', 'C4', '5.9.0', 'D4', '0.6', 'B-1', '10.2.5', '10.2.5', 'B-1', '5.10', 'D4', '5.10', '10.2', '1.2.6', 'F3', '10', 'C#6', '2', '2.5.9', 'D2', '0.1.6', 'F4', 'A4', 'D5', 'D2', 'F4', '2.6', '9.2', '5.9', '9.0', 'F5', 'F2', 'D2', '7.10.2', 'G2', '7.10.2', '0.6', 'G2', 'B-4', '10', 'D4', 'G4', '7.10.2', '2.6', 'A4', 'C#6', '5.9', 'F2', '2.5', '5', '0.1.6', '2.5.9', 'D2', 'D2', 'D4', '2.5.9', 'F4', 'C2', 'F4', '9.2', '2.5', '2.6', 'A4', 'F2', 'F2', 'C#6', '7.10.2', '0.1.6', '7', '7.10.2', '7.10.2', '7.10.2', '2.6', '2.5.9', '9', 'A4', 'F2', 'C#6', 'F2', '2.5.9', '4.7', 'A2', '0.1.6', 'A1', '1.4.7', 'E4', '9.1.4', 'A3', 'F4', '5.9', 'D4', 'D4', '1.2.6', 'D3', 'D2', '0', 'C#6', '7', '0.1.6', '5.7.11', '5.11', 'G4', 'A4', 'G4', '1.2.6', '0.5', '5.9.0', '9', '9.0', 'A2', 'G4', 'E4', 'B2', '0.1.6', '11.2.5', '11.2.5', '11', 'E4', 'D4', '1.2.6', '0.4.7', 'C3', '7.0', 'E4', 'E3', 'D2', 'D2', 'D2', '1.2', 'G3', '5.10', '10', '0.1', 'D4', 'F3', '10.2', 'F4', '10.2', 'F4', '2.5', 'B-4', 'F3', 'A4', 'B-3', 'D4', 'F4', 'A2', 'A1', '0.1.6', 'A1', 'A4', 'A4', 'A3', '9.1.4', 'C#4', '2.6', 'E4', '7.9', '9', 'C#6', 'D3', '0.1.6', 'D2', 'D2', 'F4', 'E4', 'D4', 'A3', 'C2', '2.5.9', '2.6', 'D4', 'F4', 'A3', 'A1', 'A2', '0.1.6', 'A1', 'A4', 'B-4', 'A3', '2.6', '9.1.4', '1.4', 'G4', 'A4', '9', 'C#6', 'A3', 'F4', '0.1.6', 'D3', 'D2', 'D2', 'E4', 'D4', 'A3', '2.5.9', '2.6', '2.5', 'D2', 'A3', 'B-3', 'D4', '0.1.6', 'G2', 'G1', 'G1', 'G4', 'G3', '2.6', '2.5', '7.11.2', 'B3', '7', 'C#6', 'G3', 'C3', '0.1.6', 'C2', 'E-4', 'C2', 'D4', 'C4', 'G3', '2.6', '0.3.7', 'E-4', 'C4', 'G3', '0.1.6', 'G4', 'G1', 'G2', 'G1', 'E-4', 'F4', 'G3', 'D4', '2.6', '7.11.2', '11.2', 'D4', 'G1', 'G1', 'C#6', 'G3', 'C4', '0.3', '0.3', '0.1.6', 'C2', 'C2', 'G3', 'C#6', 'F1', '0.5', '0.1', 'F2', 'F3', '0.1', '0.5', 'F1', 'F3', 'F1', 'F2', 'F#3', 'F#3', 'F#3', '10.0', '10.0', '3.7', 'B-3', 'E-4', 'F#3', '3.7', 'C4', '10.3', '10.0', '3.7.10', 'G4', 'E-4', 'F#3', 'C2', '0.1', '0.5', 'F3', 'F3', '5', '0.5', '0.1', 'F#3', 'F#3', 'F#3', '6.10.0', '3.7.10', '3.7', 'B-3', '0.3', 'G4', 'B-3', 'G3', '10.3', '6.10.0', '0.3', 'E-4', 'C2', '0.1', '0.5', 'F2', 'F3', 'F3', '0.1', '5', '0.5', 'F1', 'B-3', 'C4', '3.7', 'E-4', 'G#4', 'B3', 'D4', 'E4', 'A4', 'E-4', 'F4', 'C4', 'A4', 'C5', 'F4', 'F4', '0.3.7', 'C5', 'C5', '0.3.7', 'B4', 'B-4', '0.3.7', 'G4', '0.3.7', '2.5.8.10', '2.5', 'F5', 'D5', '2.5.8.10', 'E-5', 'C5', '2.5.8.10', '8.10', '10.2.5', 'G2', 'C#6', 'F2', 'C#6', 'F4', 'G4', '10.0.1', '0.3.7', 'C2', 'C3', 'C5', '0.3.7', 'C5', '0.3.7', 'G4', '0.3', 'G4', '10.2', '5.8.10', 'D5', 'B4', '2.5.8.10', 'C5', 'B-2', '0.2', '3.7.10', 'C2', 'E-5', 'B-2', 'C2', '2', '3.7.10', '0.1', '7', 'F5', '5', 'C#6', '3.7.10', '0.1.6', 'E-2', 'E-2', 'F5', 'E-5', 'G4', 'E-2', '1.2.6', '10.3', 'C#6', 'E-3', 'B-4', 'F5', 'D2', '0.1.6', '7.10.2', 'D2', 'B-4', 'D5', '1.2.6', 'D2', '7.10', 'D4', 'D3', 'B-4', '0.1.6', 'C#2', '7.10.1', 'C#2', 'E-5', 'C#5', 'F5', '10.1', '1.2.6', 'C#2', 'G4', 'G5', 'C#3', 'C5', 'G#1', '8.0.3', '0.1.6', 'G#1', 'E-5', 'G#4', 'C5', 'F5', '3.8', '1.2.6', 'G#1', 'C4', 'C#6', 'G#2', 'B4', '0.1.6', '3.8', 'G#1', 'G#1', 'B3', 'G#4', 'E-4', '1.5', '8', '1.2.6', '11.3', 'G#3', '8.11', 'C#6', 'G#2', 'E-4', 'B-4', '3.7', '7.10', '0.1.6', 'B-1', 'B-1', 'E-4', 'G4', '7.10', 'C5', '1.2.6', '10.3', '7.8', 'E-4', 'E-5', 'G4', 'C#6', '5.10', 'F2', 'B-1', '8.0.3', '0.1.6', '3.7', 'B-2', 'B-4', 'B-4', 'E-5', 'B-1', 'C4', '7.8', '1.2.6', 'E-4', 'E-5', 'F2', 'C#6', 'F1', 'G5', 'B-4', '0.1', '5.8.10', '10.2', 'G2', 'A2', 'G2', '3.7.10', '0.1', '3.7.10', 'B-1', 'B-1', '7.10', 'E-5', 'E-5', '7.10', '10.3', '2.6', 'G5', 'B-4', 'B-2', 'C#6', 'B-1', '3.7', '0.1.6', 'B-4', '10.0', '8.0.3', 'B-1', 'C2', 'F5', '1.2.6', 'D5', '8.10', '0.3', 'G#5', 'F2', 'F1', 'C#6', '0.1', 'B-1', '5.8.10', 'D4', 'F3', 'F3', 'F#3', 'G3', '0.1.6', '2.5.7', 'G1', '7', '11.2', 'G4', '11.2.5.7', 'D2', '7.11.2', '2.6', 'F4', 'G1', 'C#6', 'G1', '0.3.7', 'C2', 'C3', 'C2', '0.1.6', 'D4', 'C4', 'C2', '0.3', 'G4', '1.2.6', '0.3.7', '0.3', 'G3', 'A4', '0.3', 'C#6', 'G4', '0.1.6', '11.2.5.7', 'G2', 'G1', 'G1', 'G4', '2.6', '11.2.5', '2.7', '7.11.2', 'D2', 'G4', '7', 'F4', 'C#6', 'E-4', 'C3', '0.1.6', '0.3.7', 'C2', 'C2', 'D4', 'C4', '2.6', '0.3.7', '0.3.7', '0.3', '7.0', 'A3', 'C#6', 'C2', 'B-3', '10.2.5', '0.1.6', 'B-1', 'B-2', 'B-2', 'B-4', '10.2.5', 'B-3', 'B-3', 'G2', '10.2.5', '2.6', '10.2.5', '2.7', '8.10', 'G#3', '5.10', '3.7.10', '3', '0.1.6', 'E-2', 'B-3', 'G4', 'E-2', 'G3', 'F3', 'G3', 'C2', '2.6', '3.7.10', '3.7.10', 'E-4', 'E-2', 'C#6', '0.1.6', '10.2.5', 'B-2', '2.5', 'B-3', 'B-4', 'B-2', 'B-1', 'B-3', 'C4', 'C#6', 'G2', '5.10', '10.2.5', '1.2.6', 'D4', 'A3', 'G#2', 'G#3', 'B-1', 'F2', '3.7', '0.1.6', 'E-2', '3.7', 'E-3', '7.10', 'G3', '7.10', 'E-2', 'B-3', 'E-4', 'C2', '10.3', 'G4', 'B-3', 'E-4', '1.2.6', '3.7.10', '3.7.10', 'F#4', '3', 'C5', '3.6', 'A4', 'B-4', '0.1.6', 'D2', 'D3', '6.9.0', 'D2', '0.2', 'A4', '2.6.9', '9.0.2', 'F#4', '1.2.6', 'G4', 'D2', 'A4', 'D2', 'D2', 'C#6', '7.10.2', 'G2', '0.1.6', '7', '10.2', '10.2', '9.0', '5', 'F2', '1.2.6', '0.5', 'A3', 'F4', 'D2', '10.2.5', '10', '10.2', 'F4', 'F4', 'B-2', 'F3', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'F4', '5.9', '0.5', 'C3', 'F2', 'A4', 'C5', 'C4', 'C4', 'C2', '0.5', '7.9', 'E-4', '5', '0.5', '7.9', '0.5', '0.5', 'B-3', '7.9', '0.5', 'A3', '7.9', 'F1', 'C3', 'F2', 'F1', 'F2', '7.9', '5', '10.1.5', '10.1', '5', '10', 'B-3', 'D2', 'F4', 'B-1', 'F3', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'C4', 'D2', 'F3', '0.5', '0.5', '5.9', 'C4', 'C2', 'A4', '0', 'E-3', '7.9', '0.5', '5', 'F4', '0.3', '0.5', '7.9', '0.5', 'B-2', 'G#4', '0.5', '7.9', '7.9', '0.5', 'G#2', 'C6', 'F4', '0.5', '7.9', 'F2', '7.9.0', '5', 'F1', 'F2', '7.9', '5', 'F2', '5', '7.9', 'F2', '2', 'C4', '0.5', '5.9', '5.9.0', 'F2', 'C2', '0.5', '5.9.0', '5.9', '2.5', '10.2.5', '10.2.5', 'B-1', '0.6', 'A1', 'B-3', 'F4', 'B-3', '5.10', 'D4', '9.10', '2.6', '0.2.5', '10.0.5', 'A4', 'B-2', 'B-1', 'C#6', '2.5', '2.7', '2', '0.1.6', 'D2', '5.7.9.10', '2.5.9', 'F4', '5.9.0', '2.6', 'F3', 'F4', '9.2', '2.5', 'F4', 'F4', '0.2.5', 'C4', 'B-3', '0.6', '10.2', '2.7', 'G4', '10.2', 'G3', '2.7', 'B-4', 'G4', 'B-4', '10.2', '7.10.2', '2.6', 'A4', '0.5', '0.5', '0.2.5', 'C#6', 'A4', '2.5.9', '2.5.9', '2', '0.1.6', 'A2', 'D4', '2.5.9', 'F4', '9.2', 'F4', 'C2', '0.5', '2.5.9', '2.6', 'A3', '1.6', '1.5.9', 'C#6', 'F2', '2.7', '10.2', '7.10.2', '7.10.2', 'G2', '0.1.6', 'G2', '7.10.2', 'B-4', '7.10', 'D4', '2.6', '7.10.2', '2.5.9', 'A4', 'A4', 'F2', '2.5.9', '4.7', 'A2', '0.6', '1.4', 'C#4', 'A2', 'A3', 'E4', 'A3', '4.9', 'C#4', 'F4', 'C2', 'D4', 'D4', '5', 'A3', '6.9', '9.2', '2.6', 'D2', 'F#4', 'C2', 'C#6', '5.7.11', '0.1.6', '7', '5.7.11', 'B3', 'D4', 'A4', 'G4', '9.0', '5.9', '5.7.9.0', '2.6', 'A2', 'C4', 'G4', '0.6', '11.2', '11.2.5.7', '2.5', 'B2', 'B1', 'E4', 'G4', 'C4', '0.4', '0.4.7', 'E4', 'G3', 'D4', 'C3', '0.4.7', '2.6', 'E3', 'D2', 'D2', 'D2', '1.2', 'G3', '0.1', 'F3', '10.2.5', 'B-1', '10.2.5', '10.2.5', 'D5', 'B-4', 'F4', '2', 'B-4', 'F3', '5.10', 'F5', 'D5', 'B-3', 'D4', 'F4', 'A4', 'A1', '0.1.6', 'A2', 'A1', 'A4', 'A4', 'A3', '9.1.4', '2.6', '1.4', 'A4', '9', 'C#6', 'F4', '0.1.6', 'D3', 'D2', 'D2', 'E4', 'D4', 'A3', 'C2', 'F4', '2.6', '2.5', 'D4', 'A3', 'A3', 'G3', '0.1.6', '7', 'G2', 'G4', '7', '5.7', '7.11.2', '2.6', 'B3', 'D4', 'C#6', 'F4', '7', 'G1', 'E-4', '0.1.6', '0', 'C3', 'D4', 'C4', 'C2', 'G3', '2.6', '4.7', '0.4', 'C4', 'G3', 'D2', 'D2', 'D2', '7.0', '0.4.7', 'D2', 'C2', 'C2', 'F3', 'D4', '0.1', '5.10', 'B-3', 'B-1', 'B-1', '10.2', '2.5', 'C4', '0.3.7', 'D4', '10.2', '2.5', 'F3', 'B-3', 'D4', 'F4', 'A3', '0.1.6', '5', 'C4', 'A4', 'F1', 'A3', 'F4', 'F3', 'F4', '1.2.6', '9.0', 'F3', '0.1.6', 'D4', 'B-3', 'C2', 'F3', 'B-3', 'G3', 'E-4', 'G3', 'F2', '1.2.6', 'B-3', 'E-4', 'G4', 'A1', '0.1.6', 'F1', 'F1', 'F2', '5.9', 'C2', 'F4', '9.0', '1.2.6', 'F4', 'F4', 'D4', 'F1', 'F3', '10.2', '10', 'B-3', '0.1.6', 'C2', 'F3', '7.10', '7.10', '1.2.6', '7', 'C2', '3.7', '3.7', 'G3', 'E-4', 'G4', 'B-3', '0.1', 'E-4', 'B-1', 'G3', 'B-3', 'F1', 'C#6', '5', 'C2', 'F3', '1.2.6', 'F4', 'F3', 'F1', '10', '0.1.6', 'F3', 'B-1', 'D4', '7', '1.2.6', '10.3', 'G1', 'G3', 'G3', 'F1', 'F1', '0.1.6', 'F2', 'F3', '2.6', 'F4', 'F3', 'F1', 'B-1', '10', '10.2', 'B-3', 'F#2', 'C2', '10.1.3', '3.7.10', '0.1', 'E-5', 'G5', '10.3', '3.7.10', '0.1', 'G5', 'E-2', 'C#4', 'C#6', 'E-2', '3.7.10', '0.1.6', 'E-2', '3.7.10', '0.1.6', 'E-2', 'C2', 'E-2', '3', '10.1', '3.7.10', '3', '3.7.10', '0.1', '3.7.10', 'E-4', '10.1.3', 'G4', '0.1', 'E-2', 'C#6', '3.7.10', '0.1.6', 'E-2', '0.1.6', '3.7.10', 'E-2', 'E-2', '0.1', 'E-4', '10.1.3', '3', '0.1', '3.7.10', 'G5', '3.7', '10.3', '0.1', 'E-2', 'G4', '3', '10.1', 'C#6', 'E-2', '3.7.10', '0.1.6', 'E-2', '3.7.10', '0.1.6', 'E-2', '0.1', '10.1.3', '3', '3.7.10', '0.1', '3.7', 'G4', '10.3', '3', 'B-5', 'E-2', '0.1', 'E-5', '3.7', '10.1', 'G5', 'F5', 'E-2', 'C#6', '3.7.10', 'E-2', '0.1.6', 'E-5', 'B-5', '0.1.6', '3.7.10', 'G5', 'F5', 'C2', 'E-2', '10.1', '3', '0.1', '3', '3.7.10', 'G5', 'E-5', '3', '10.1', 'B-5', '3.7', '0.1', 'G4', 'E-2', 'B-3', '3', 'G5', 'F5', 'E-2', 'C#6', '0.1.6', '3.7.10', 'E-2', 'E-5', 'B-5', '3.7.10', '0.1.6', 'E-2', 'G5', 'F5', '0.1', '10.1', '3', '0.1', '3.7.10', '3', 'E-5', 'E-5', 'G5', '10.3', '0.1', '10.3', '3', 'G4', 'E-2', 'B-5', 'C#4', 'G5', 'F5', 'E-2', '0.1', '10.3', '3.7', '0.1.6', 'E-5', 'B-5', '0.1.6', '3.7.10', 'G5', 'F5', 'E-2', 'C2', '10.1.3', '0.1', '3.7', '3.7.10', 'E-5', '3', 'B-5', 'G5', '10.3', '0.1', '10.1.3', 'E-2', 'E-4', 'G4', 'G5', 'F5', 'E-2', '0.1.6', '3.7.10', 'E-2', 'E-5', 'B-5', '0.1.6', '3.7.10', '3.7.10', 'G5', 'F5', '0.1', 'E-2', '3.7.10', '7.10.1.3', '0.1', '3.7.10', '3', 'E-5', 'E-5', 'E-4', '0.1', 'B-5', 'E-3', '10.3', 'E-2', '7.10.1.3', 'G4', 'G5', 'F5', 'E-2', '3', '3.7.10', '0.1.6', 'E-5', 'B-5', '0.1.6', '3', '3.7.10', 'G5', 'F5', '0.1', 'E-2', '10.1', '3.7.10', '3.7', '0.1', '3', '3.7', 'E-5', 'E-5', '3', 'B-5', '3.7.10', '10.1', '0.1', 'E-2', 'G4', 'G5', 'F5', 'C#6', 'E-2', 'E-2', '0.1.6', '3.7.10', 'E-5', 'B-5', '0.1.6', '3.7.10', 'E-2', 'G5', 'F5', 'C2', 'E-2', '3', '0.1', '3.7.10', '7.10.1.3', 'E-5', 'E-5', '3', 'B-5', '0.1', 'E-2', 'G4', '10.3', '10.1', 'G5', 'G5', 'F5', 'E-2', '0.1.6', '3.7.10', 'E-5', 'B-5', 'E-5', '0.1.6', '3.7.10', 'G5', 'F5', 'C2', 'E-2', '7.10.1.3', '0.1', 'E-5', 'E-5', 'B-5', '3', '7.10.1', '3', '0.1', 'E-2', '3', 'G5', 'F5', 'E-5', 'B-5', 'G5', 'F5', 'E-5', 'E2', '0.4', '0.4', '0.1.4', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', 'B-2', '9.0.4', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', '0.4.8', 'E2', '0.4.8', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '8.0', '0.4.8', 'E2', '0.1.5', '0.1.5', '0.1.5', '0.1.4', 'E2', '0.5', '0.5', '0.5', '0.5', '0.1.5', '0.1.5', '0.1.5', '0.1.4', 'E2', 'C#2', '10.0.4', '0.1.5', '0.1.5', '0.1.5', '0.1.5', '0.1', '0.4', 'E2', '9.0.1', '0.4', 'E2', '0.1', '0.4', 'E2', '9.0.1', '0.4', 'E2', '0.1', '0.4', 'E2', '9.0.1', '0.4', 'E2', '0.1', '0.4', 'E2', '9.0.1', '0.4', 'E2', '0.1', '0.1', '0.1', '0.1', '0.1.4', '0.1.4', '0.1.4', '0.1.4', 'C#3', '9.0.1.4', 'E3', '0.1.4', '1.4', '0.1', '0.1.4', '1.4', '0.1', '0.1.4', '1.4', '0.1', '0.1.4', '1.4', '9.0.1', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', '9.0.1.4', 'F#3', 'F#3', '6.8', '6.8', '6.8', '6.8', '6.8', 'F#3', '0.1.6', 'F#3', 'F#3', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '6.8.0', '6.8', '4.6.8.0', '6.8', '6.8.0', '6.8', '4.6.10.0', '6.8', '0.4.8', '4.8', '0.4.8', '10.0.4', '9.0.4', '9.0.4', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', 'A3', 'C#3', '4.9', 'E2', 'E2', 'E2', '0.1.4', 'F3', 'F3', 'F3', '4.5.9.0', '0.1.5', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', '0.1.5', '0.1.5', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', '0.1.5', '0.4.5', '0.4.5', '0.4.5', '0.4.5', '0.4.5', '0.4.5', '0.4.5', '0.4.5', '0.1.5', 'G#2', 'G#2', '8.0', '1.4', '8.0', 'G#2', 'G#2', '0.1.5', 'G#2', 'G#2', '8.0', '1.4', '8.0', 'G#2', 'G#2', '0.1.5', 'G#2', 'G#2', 'G#2', '1.4', '8.0', 'G#2', 'G#2', '0.1.4.5', 'G#2', 'G#2', 'G#2', '1.4', '8.0', 'G#2', 'G#2', '9.0.1.4', 'E3', '0.1.4', '1.4', '0.1', '0.1.4', '1.4', '9.0.4', 'B-2', '9.0.4', 'B-2', '0.1.4', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', '9.0', 'E2', '0.4', 'E2', 'E2', 'C2', 'E2', '1.4', '0.4', 'E2', 'E2', '0.1.4', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', 'E2', '9.0', 'E2', '0.4', 'E2', 'E2', 'C2', 'E2', '1.4', '0.4', 'E2', 'E2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', '4.7.10.0', 'G#2', '0.4.7.8', 'G#2', 'B-2', 'G#2', 'G#2', 'G#2', '0.3.4', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', '0.1.4', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', 'E-3', '0.1.4', '0.1.4', '6.9.0.1', 'F#3', 'F#3', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.9.0.1', 'F#3', 'F#3', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.9.0.1', 'F#3', 'F#3', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', '6.8', 'F#3', '4.6.8.0', 'F#3', '6.8', 'F#3', '6.8.0', 'F#3', 'F#3', '6.8', '4.6.10.0', 'F#3', '6.8', 'F#3', '0.1.4', '1.4', '0.1.4', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '5.9.0', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '7.10.2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', '5.9', 'C6', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', '5.9.0', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '7.10.2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', '5.9', 'C6', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', '7.9.0', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'D5', 'C2', 'B-4', 'F4', 'C2', 'D4', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'F4', 'C2', 'B-4', '0.5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'D6', 'B-5', 'C2', 'F5', 'D5', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', 'C5', '5.9', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', '5.7.9.0', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'B-5', 'C2', 'A5', 'G5', 'C2', 'D5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'F4', 'C2', 'B-4', 'C5', 'F5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'D5', 'B-4', 'C2', 'F4', 'D4', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', 'C6', '5.9', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '9', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '8', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', 'A5', '0.5', 'C2', 'B-5', 'A5', 'C2', 'C2', 'C2', 'A5', '0.5', 'C2', 'B-5', 'A5', 'C2', 'C2', 'C2', 'C5', 'G5', 'F5', 'C2', 'A5', 'G5', 'C2', 'E5', 'C2', 'C2', 'D5', 'F5', 'G5', 'C2', 'A5', 'G5', 'C2', '0.4', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', '5.9', 'C5', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C5', 'F5', '7.9', 'C2', 'C6', '5.7.9', 'C5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'B-5', 'C2', 'A5', 'G5', 'C2', 'D5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'F4', 'C2', 'B-4', 'C5', 'F5', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'D5', 'B-4', 'C2', 'F4', 'D4', 'C2', 'C2', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', 'C6', '5.9', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '9', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '8', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', 'A5', '0.5', 'C2', 'B-5', 'A5', 'C2', 'C2', 'C2', 'A5', '0.5', 'C2', 'B-5', 'A5', 'C2', 'C2', 'C2', 'C5', 'G5', 'F5', 'C2', 'A5', 'G5', 'C2', 'E5', 'C2', 'C2', 'D5', 'F5', 'G5', 'C2', 'A5', 'G5', 'C2', '0.4', 'C2', 'C2', '10.2.5', 'C2', '5.9.0', 'C2', '5.9', 'C5', 'C2', 'C2', '7.10', 'C2', '5.9', 'C2', '5.9', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '9', 'C2', 'C2', 'C2', 'C2', '5', 'C2', 'C2', '7', 'C2', 'C2', '8', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', 'C2', 'C2', '2', 'C2', 'C2', '4', 'C2', 'C2', '5', 'C2', 'C2', '5.9.0', 'C4', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', '5.7.0', 'C2', '5.9.0', 'C2', '10.0', 'F5', 'C4', 'C2', 'C2', 'C2', 'C2', 'C5', 'D5', 'F5', 'C4', '5.9', '5', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'B4', 'A4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'D5', 'D5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'E4', 'E4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'A4', 'A4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'D5', 'D5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'A4', 'A4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'A4', 'A4', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'E5', 'E5', 'D5', 'D5', 'D5', 'D5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'F#5', 'F#5', 'A5', 'A5', 'G5', 'G5', 'E5', 'E5', 'D5', 'D5', 'D5', 'D5', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'G4', 'G4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'B4', 'B4', '7', '11', '2', '7', '11', 'E5', 'E5', 'F#5', 'F#5', 'A5', 'A5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'G4', 'G4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D5', 'D5', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'A4', 'A4', 'D5', 'E5', 'E5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'E5', 'E5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', '4.7', '4.7', '6.9', '6.9', '7.11', '7.11', '7.11', '7.11', '7.11', '7.11', '6.9', '6.9', '6.9', '6.9', '4.7', '4.7', '2.6', '2.6', 'D5', 'E5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'F#5', 'F#5', 'A5', 'A5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'B5', 'B5', 'C6', 'C6', 'B5', 'B5', 'A5', 'A5', 'G5', 'G5', 'E5', 'E5', 'D5', 'D5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'E5', 'E5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'E5', 'E5', 'D5', 'D5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'F#5', 'F#5', 'G5', 'G5', 'F#5', 'F#5', 'A5', 'A5', 'G5', 'G5', 'C6', 'C6', 'B5', 'B5', 'C6', 'C6', 'B5', 'B5', 'A5', 'A5', 'G5', 'G5', 'B-4', 'B-4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'B-4', 'B-4', 'A4', 'A4', 'G4', 'G4', 'G4', 'G4', 'F#3', 'G3', 'A3', 'A3', 'B3', 'C#4', 'D4', 'F#3', 'G3', 'A3', 'A3', 'B3', 'C#4', 'D4', 'F#3', 'G3', 'A3', 'A3', 'B3', 'C#4', 'D4', 'E4', '7', '10', '0', 'C2', '0', 'C3', 'G3', 'C4', 'G4', 'E-4', 'C4', '2.5.7.10', 'C4', '7', 'B-3', 'D4', 'F4', 'G3', '7', 'E-3', 'B-2', '10', '7.10', '0', 'C3', 'C2', '0', 'C3', 'G3', 'C4', 'G4', 'C5', 'E-5', 'C6', '3.7', 'G5', 'C3', 'C4', 'E-5', 'G3', '2.5.7.10', '2.7', '5.7.10', '7', '7.10', '7.10', 'D4', 'F4', '7.10', '7.10', 'G3', 'B-3', '7.0', 'B-3', 'C3', 'C2', '0', 'C3', 'G3', 'C4', 'G4', 'E-5', 'C6', '0', '0', 'G3', 'E-4', 'G5', '0', 'C3', 'G3', 'C4', '0.3', '0', '2.5.7.10', 'G2', 'G3', 'B-3', 'D4', 'F4', '7', 'G3', '2', 'G2', 'B-3', '0', '10.2', '0', 'C2', 'G3', 'C4', 'C3', 'E-4', 'E-5', '0', 'G3', 'G4', 'G3', 'G2', 'B-3', 'D4', 'F4', 'G2', 'G3', '10', 'B-3', '10.2', 'D4', '2.5.7', '3', 'E-3', 'E-2', '3', 'G3', 'B-3', 'E-3', 'G3', 'E-4', 'G4', 'G4', 'B-3', 'E-4', 'G3', 'B-3', '8.0.3', 'G#4', 'G#4', 'G#3', 'C4', 'E-4', 'G#4', 'E-4', 'B-3', '10', 'G#3', '2', 'C4', '3', 'E-2', 'E-2', 'E-3', 'G3', 'B-3', 'G4', 'G4', 'E-4', 'E-4', 'E-3', 'B-3', 'B-3', '8.0.3', 'G#4', 'B-4', 'E-2', 'G#3', '0.3', 'G#4', 'E-4', 'B-3', '10', 'E-3', 'E-3', '2', 'G#3', '10.3', 'C4', '3', '11.1', '3.7.10', 'G3', 'E-4', '3', '5', '10.2', '9.11', '2.5', 'F3', 'D4', '5', '7', '0.3', '11.1', 'C2', 'G3', 'C4', 'G4', '7', '9.0', '5', '9.11', 'F3', '5.9.0', 'A4', 'A2', '10', 'G2', 'B-1', '11.1', 'B-2', 'B-3', '2.5', 'B-4', '10', '7', '5', 'G2', 'G2', 'G2', 'F3', 'G2', 'G2', '5.10', '5.10', 'G2', 'D4', 'G2', 'F4', '10.2', '2.5', 'G2', 'B-2', 'D2', 'C4', 'E-4', '8.0', 'D4', '8.0', 'C3', 'B-2', '7.9', 'B-2', 'B1', 'B-2', 'G2', '10.2.5', 'B-4', 'B-2', 'B1', 'B-2', 'D2', '10.3', '8.0.3', '8.0', 'B-2', 'C3', 'B-2', '7.9', 'B-2', 'B1', '5.10', 'G2', 'B-2', '2.5', 'B-4', 'B-4', 'B-2', 'B1', 'B-2', 'D2', '8.10', 'C4', 'E-4', '0.4.8', 'B-2', 'C3', 'B-2', '7.9', 'B-2', 'B1', 'B-2', 'G2', '10', 'F4', '10.2.5', 'B-2', 'B1', 'C3', '9.2', '0.4.7', 'C4', '0', 'G4', 'C4', 'F1', '11.1', '5', 'F3', 'F4', '9.0', 'F5', '0.5', 'E-3', 'F4', 'F2', '2.3', 'F3', '4.5', 'E-3', 'F4', '7', '11.3', 'G2', '5', 'A1', '11.3', 'A2', 'A4', 'C5', 'E5', '4.9', '4', 'E-3', 'E-3', 'E4', '0', 'A2', '2.3', 'A3', 'A1', 'E-3', 'A4', '9', 'C2', '11.3', 'E3', 'C5', 'D2', '11.3', 'D3', 'F4', 'A4', 'D5', '9.2', '2', 'E-3', '9', 'E-3', 'F4', 'D2', '2.3', 'D4', 'D2', 'E-3', 'A4', 'C2', '11.3', 'A3', '10.2.5', '11.4', '10', 'F4', 'E-3', '11.3', '7.0', '1.2', 'C4', '4.7', 'C5', 'E-3', '11.3', '0', '5', '7.11', 'E2', 'F3', 'F4', 'A4', 'F5', '0.5', 'E-3', 'E-3', 'F4', 'F2', '2.3', 'F3', '5.7', 'E-3', 'F4', '7.8.9', '11.3', '0', 'A1', '11.3', 'A2', 'A3', 'A4', '9.0.4', '9', 'E-3', '9', 'E-3', 'E4', 'A2', '2.3.6', 'A3', 'A1', 'E-3', 'E4', '2', 'A2', '11.3', 'E3', '4', 'B-1', '11.3', 'B-2', 'F4', '10.2', '10', '2', 'E-3', 'E-3', 'F4', '2.3.5', 'B-3', 'A1', '11.1', '9', 'G1', '9.11', 'G3', '7', '6', '11.1', 'F#3', 'F#4', 'A4', 'C5', '3.5', '3.6', '11.3', '3', '2.3.5', '2', '11.3', 'F#4', 'D5', 'D4', '3.5', '7.10', '11.2.4', '2.7', '2.7', 'B-4', '7', 'G3', '2', 'E-3', 'E-3', '7.0', '2.3', 'D4', 'A3', 'C4', 'E-3', '2', 'G1', '11.3', 'D4', 'G4', 'D4', 'G1', '11.3', 'G3', '0.4', 'E3', '0', 'E-3', 'E-3', 'G2', '2.3', '4.10', 'B-4', '11.3', 'G4', 'A1', '3.7.11', 'G3', '10', '5.10', '11.1', 'B-3', 'B-4', 'F5', 'D4', 'F4', 'E-3', 'E-3', 'F4', '5.9.10', '2.3', 'E-3', 'C4', '10.1.5', '11.3', '1', '10', '5.9', '11.3', '9.1', 'B-4', '1', 'F5', 'E-3', 'E-3', '5.7.8', '2.3', '7', 'E-3', 'E-3', '5.9', '7.11', 'F3', 'A4', 'C5', '0.5', '5.9.0', '5.9', 'E-3', 'E-3', 'F2', 'G2', '2.3', 'F4', '5', 'F2', 'E-3', 'F1', '11.3', '5.10', 'A4', '7.9', '11.1', 'G3', '5.7.0', 'E-5', '7', 'G3', 'E-4', 'E-3', 'E-3', 'G2', 'A2', '2.3', '5', 'G2', 'E-3', 'F4', 'G1', '11.3', '10', '8.0', '9.11', '8.0', 'G#4', 'E-5', '8', 'E-3', 'E-3', 'G#2', 'B-2', '2.3', 'G#4', '5', 'G#2', 'E-3', 'C5', 'G#1', '11.3', 'E-5', '10', 'B-1', '11.1', 'B-2', 'B-3', 'F4', 'B-4', '10.2.5', '10.2', '8', 'E-3', 'F3', 'B2', 'C3', 'D2', 'B-3', '5', 'B-2', '7.11', 'F4', '5.10', '9.11', '7.10.0.3', '11.2', '7', '0', '9.11.2', '7.0', 'B-4', 'E-5', '7', '3', '0', 'F2', 'F3', '5', 'F4', 'G#4', 'F3', '0.5', 'G#4', 'F4', 'C3', 'B-3', 'D4', 'B-3', 'F4', 'D4', 'C3', 'B-3', 'F2', 'F3', 'F4', 'G#4', 'F4', 'F3', 'G#4', 'C5', 'F5', 'G#4', 'F2', 'F3', '10.0.2.5.7', 'F3', 'C3', 'G3', 'B-3', 'D4', 'F4', 'G3', 'B-3', 'C3', 'D4', 'F2', 'F3', 'F4', 'G#4', 'F4', 'F3', 'G#4', 'C5', 'F5', '5.8', 'F2', 'F3', '10.0.2.5.7', 'F3', '10.0.2.5', '2.5.7.10', '7', '10', '0', 'C2', 'C2', '0', 'E-3', 'C3', 'G3', 'C4', 'G4', 'F#2', 'E-4', 'F#2', 'F#2', 'C4', 'F#2', '2.5.7.10', '3.6', '7', 'B-3', 'D4', 'F4', '7.10', '3.6', 'F5', 'E-3', 'D5', '10', 'F#2', '10.0', 'B-4', 'E-3', 'F#2', 'C2', '0', 'C3', 'G3', '6.8', 'C4', 'G4', 'C5', 'E-5', 'C6', 'G4', 'F#2', '3.7', 'G5', 'B-4', 'F#2', 'C3', 'C4', 'E-5', 'C5', 'F#2', 'F4', '3.6', '2.5.7.10', 'G4', 'E-3', '5.7.10', '2.5', '7', '3.6', '7.10', 'D4', 'F4', '7.10', 'G3', 'B-3', '7.0', '3.6', 'B-3', 'F#2', 'C2', '0', 'F#2', 'C3', 'C4', 'F#2', 'F#2', 'G3', 'E-4', '0.3.6.8', 'C4', 'G4', 'E-5', 'C6', 'G4', '0', '0', 'E-3', 'G3', 'E-4', 'G5', 'B-4', '0', '3.6.7', '0', 'C4', '0.3', '0', 'F#2', '2.5.7.10', 'F4', 'D4', '3.6', 'G2', 'G3', 'B-3', 'D4', 'F4', '7', 'F#2', 'G3', '2', 'G2', 'B-3', 'F#2', 'D4', '0', '10', '0', 'F#2', 'C2', 'E-3', 'G3', 'B-3', 'F#2', 'F#2', 'C4', 'C4', '3.6', 'C3', 'E-4', 'E-5', 'E-4', 'F#2', '0', 'G4', 'F#2', 'G3', 'G4', 'F#2', 'G3', 'E-4', 'F#2', '3.7', 'B-3', 'D4', 'F4', 'B-3', 'G2', 'G3', 'D4', '10', 'B-3', '7.10.2', 'D4', '2.3.5.7', 'B-3', 'E-3', '11.1', 'E-2', '3', 'G3', 'E-2', 'B-3', '2.3', 'E-3', 'G3', 'B-3', 'E-4', 'G4', 'G4', 'E-2', 'E-3', 'B-3', 'E-4', 'E-2', '11.3', 'G3', 'B-3', '8.0.3', '11.3', 'G#4', '0.3', 'E-3', 'G#3', 'E-4', 'G#4', 'E-4', 'E-3', 'B-3', '10', '2.3', 'B-2', 'E-3', 'G#3', '2', '0.3', '3', 'E-3', 'E-3', '11.3', 'E-2', 'E-3', 'E-3', '3', 'G3', '2.3', 'B-3', 'G4', 'G4', 'E-2', 'E-3', 'E-4', 'E-4', 'E-2', '11.3', 'E-3', 'B-3', 'B-3', '8.0.3', '11.3', 'B-4', '3', 'E-3', 'G#3', '0.3', 'G#4', 'E-4', 'E-3', 'B-3', '10', '2.3', 'E-3', '2', 'G#3', 'D2', '10.3', 'C4', 'B1', '3', '9.11', '3.7.10', 'G3', 'E-4', '3', '10', '3', 'B1', '5', '2.5', '1.2', '2.5', '10.2', 'D4', '5', 'B2', '7', 'G2', 'B1', '0.3', '7.11', 'C2', 'G3', '0.3', 'G4', '0', '7', 'B1', 'B1', '9.0', '0.5', '11.1.2', 'F3', '5.9.0', 'A4', '5', 'A2', '10', 'G2', '10', '11.4', 'B-2', 'B-3', '2.5', '10', '2', '10', '7', '5', 'G2', 'G2', 'D2', 'F3', 'D2', 'G2', '5.10', '5.10', 'G2', 'D4', 'F2', 'G2', '10.2.5', '2.5', 'G2', '8.10.0', 'D2', '0', 'E-4', '10.2.5', '8.0', 'C3', '7.9', 'B1', '5.10', 'G2', '10.2.5', 'B1', 'B-2', 'D2', '10.3', '8.0.3', '8.0', '5.10', 'C3', '7.9', 'B1', 'F2', 'G2', 'B-2', '2.5', 'B-4', '5.10', 'B1', '8.10.0', 'D2', '8.10', 'C4', 'E-4', '5.10', '0.4.8', 'C3', '7.9', 'B1', '5.10', 'G2', '5', '10.2.5', 'B1', '7.0', '1.2', '0.4.7', 'C4', '0', 'B2', 'G4', 'B2', 'G2', 'C4', 'G2', 'F1', '5', '11.4', '5', 'F3', '9.0', 'F5', '0.5', 'E-3', 'F4', 'F2', '2.3', 'F3', '4.5', 'E-3', 'F4', '7', '5', '11.3', 'G2', 'A1', '11.3', 'A2', 'A4', 'C5', 'E5', '4.9', '4', 'E-3', 'E-3', 'E4', '0', 'A2', '2.3', 'A3', 'A1', 'E-3', 'A4', '9', 'C2', '11.3', 'E3', 'C5', 'D2', '11.3', 'D3', 'F4', 'A4', 'D5', '2', '9.2', 'E-3', '9', 'E-3', 'F4', 'D2', '2.3', 'D4', 'D2', 'E-3', 'A4', 'C2', '11.3', 'A3', 'D5', 'A4', '10.2.5', '11.1', '10', 'F4', 'D5', 'E-3', '11.3', 'D5', 'A4', '7.0', '2.7', 'C4', '4.7', 'C5', 'D5', 'E-3', 'C5', 'A4', 'G4', '11.3', '0', 'F4', 'D4', '5', '11.1', 'E2', 'F3', 'F4', 'A4', 'F5', '0.5', '0', '5', 'E-3', 'E-3', 'F4', 'F2', '2.3', 'F3', '5.7', 'E-3', 'F4', '7.8.9', '11.3', '0', 'A1', '11.3', 'A2', 'A3', 'A4', '9.0.4', '9', 'E-3', '9', 'E-3', 'E4', 'A2', '2.3.6', 'A3', 'A1', 'E-3', 'E4', '2.4.9', '11.3', '4', 'B-1', '11.3', '10.2', 'F4', '10.2', '10', 'E-3', 'E-3', 'F4', '2.3.5', 'B-3', 'A1', '9.11', '9', 'G1', '11.1', 'G3', 'E-4', 'D4', 'C#4', '2.6', '9.11', 'F#3', 'F#4', 'A4', 'C5', '3.5', '3.6', '11.3', '3', 'E-4', 'D4', 'C#4', '2.3.5', 'D4', '2', '11.2.3', 'F#4', 'D5', 'G4', '3.5', 'A4', '7.10.2', '11.1.2', '2.7', '2.7', 'B-4', 'B-4', 'G3', 'E-3', 'E-3', '7.0', '2.3', 'D4', 'A3', 'C4', 'E-3', '2', 'G1', '11.3', 'D4', 'G4', 'G1', '11.3', 'G3', '0.4', '7', '2', 'E3', '0', 'E-3', 'E-3', 'G2', '2.3', '4.10', 'B-4', '11.3', 'G4', 'A1', '3.7.11', 'G3', '10', '5.10', '11.4', 'B-3', 'B-4', 'F5', 'D4', 'F4', 'E-3', 'E-3', 'F4', '5.9.10', '2.3', 'E-3', 'C4', '10.1.5', '11.3', '1', '10', '5.9', '11.3', '9.1', 'B-4', '1', 'F5', 'E-3', 'E-3', '5.7.8', '2.3', '7', 'E-3', 'E-3', '5.9', '11.1', 'F3', 'A4', 'C5', '0.5', '5.9.0', '5.9', 'E-3', 'E-3', 'F2', 'G2', '2.3', 'F4', '5', 'F2', 'E-3', 'F1', '11.3', '5.10', 'A4', '7.9', '7.11', 'G3', '5.7.0', 'E-5', '7', 'G3', 'E-4', 'E-3', 'E-3', 'G2', 'A2', '2.3', '5', 'G2', 'E-3', 'F4', 'G1', '11.3', '10', '8.0', '11.1', '8', 'G#4', 'E-5', '8.0', '8', 'E-3', 'E-3', 'G#2', 'B-2', '2.3', 'G#4', '5', 'G#2', 'E-3', 'C5', 'G#1', '11.3', 'E-5', '10', 'B-1', '9.11', 'B-2', 'B-3', 'F4', 'B-4', '10.2.5', '8', 'E-3', 'F3', 'B2', 'C3', 'D2', 'B-3', '5', 'B-2', '7.11', 'F4', '5.10', '9.11', '7.10.0.3', '11.2', '7', '0', '9.11.2', '7.0', 'B-4', 'E-5', '7', '3', '0', 'F2', 'F3', '5', 'F4', 'G#4', 'F3', '0.5', 'G#4', 'F4', 'C3', 'B-3', 'D4', 'B-3', 'F4', 'D4', 'C3', 'B-3', 'F2', 'F3', 'F4', 'G#4', 'F3', '0.5', 'G#4', 'F4', 'C3', 'B-3', 'D4', 'B-3', 'F4', 'D4', 'C3', 'B-3', 'F2', 'F3', 'F4', 'G#4', 'F3', '0.5', 'G#4', 'F4', 'C3', 'B-3', 'D4', 'B-3', 'F4', 'D4', 'C3', 'B-3', 'F2', 'F3', 'F4', 'G#4', 'F3', '0.5', 'G#4', 'F4', 'C3', 'B-3', 'D4', 'B-3', 'F4', 'D4', 'C3', 'B-3', 'F2', 'F3', 'G3', 'G#3', 'C4', 'E-4']

In [ ]:
notes2 = ['6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', 'E4', '6.11', 'B2', '6.11', 'E-4', '6.11', 'B2', '6.11', 'B2', 'B3', '6.11', 'B2', 'B3', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', 'E4', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', '6.11', 'B2', 'E4', '6.11', 'B2', '6.11', 'E-4']

In [ ]:
len(notes1), len(notes2)